In [1]:
!pip install xlsxwriter

from google.colab import drive
drive.mount('/content/drive')

import sys
path_source_cod = r'/content/drive/MyDrive/Магистратура/Научная работа/Parser/Source'
sys.path.append(path_source_cod)

from OperationProgress import get_OperationProgress
from PhysicianDiaryEntry import get_PhysicianDiaryEntry
from ProtocolForEstablishingDeath import get_ProtocolForEstablishingDeath

from constants import path_HTML_history


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.5/153.5 kB 3.1 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
#----------------------------------Выгрузка документов вида "Ход операции"
get_OperationProgress()

In [ ]:
#----------------------------------Выгрузка документов вида "Дневниковая запись лечащего врача"
get_PhysicianDiaryEntry()

In [ ]:
#----------------------------------Выгрузка документов вида "Протокол установления смерти человека"
ProtocolForEstablishingDeath()

In [2]:
path_HTML_analiz = r"/content/drive/MyDrive/Магистратура/Научная работа/DataSetes/Онкология/Analyzes"
path_GeneralBloodAnalysis = r'/content/drive/MyDrive/Магистратура/Научная работа/DataSetes/Histores/Docs/Общий анализ крови Онкалогия.xlsx'
path_AVBlocad = r'/content/drive/MyDrive/Магистратура/Научная работа/DataSetes/Histores/Docs/A-В блокада.xlsx'
path_locad = r'/content/drive/MyDrive/Магистратура/Научная работа/DataSetes/Histores/Docs/A-В блокада.xlsx'


In [6]:
import sys
path_source_cod = r'/content/drive/MyDrive/Магистратура/Научная_работа/Parser/Source/'
sys.path.append(path_source_cod)


from get_name import get_name_history
from get_cod import get_cod_history

import re
from os import walk, listdir
from bs4 import BeautifulSoup
from tqdm import tqdm
import datetime
import pandas as pd
import numpy as np

#------------------------------------------------------------------------------------Выгрузка документов вида "Общий анализ крови" и "Общий анализ крови-экспрес"

def get_GeneralBloodAnalysis_docs():
    table = []
    print( "Загрузка документов вида \"Общий анализ крови\" и \"Общий анализ крови-экспрес\" начата" )

    for file in tqdm(listdir(path_HTML_analiz)):
        if '.html' not in file:
                continue
        with open(f'{path_HTML_analiz}/{file}', encoding='windows-1251') as file:
                        src = file.read()

        htmls = get_GeneralBloodAnalysis_doc(src,"<b>Общий анализ крови</b>","Общий анализ крови")

        htmls_e = get_GeneralBloodAnalysis_doc(src,"<b>Общий анализ крови_экспресс</b>","Общий анализ крови_экспресс")

        name = None
        if( len(htmls) > 0 and htmls[0]["Name"]!="-" ):
            name = htmls[0]["Name"]
        elif( len(htmls_e) > 0 and htmls_e[0]["Name"]!="-" ):
            name = htmls_e[0]["Name"]

        cod = None
        if( len(htmls) > 0 and htmls[0]["Cod"]!="-" ):
            cod = htmls[0]["Cod"]
        elif( len(htmls_e) > 0 and htmls_e[0]["Cod"]!="-" ):
            cod = htmls_e[0]["Cod"]

        if( len(htmls) > 0 and htmls[0] != None ):
          for html in htmls :
            table.append({
                    "File Name": file.name,
                    "Name": name,
                    "Cod": cod,
                    "HTML": html["HTML"],
                    "Doc": html["Doc"],
                    "Find": "Да",
                })
        else:
          table.append({
                    "File Name": file.name,
                    "Name": name,
                    "Cod": cod,
                    "HTML": "-",
                    "Doc": "Общий анализ крови",
                    "Find": "Нет",
                })

        if( len(htmls_e) > 0 and htmls_e[0] != None ):

          for html in htmls_e :
            table.append({
                    "File Name": file.name,
                    "Name": name,
                    "Cod": cod,
                    "HTML": html["HTML"],
                    "Doc": html["Doc"],
                    "Find": "Да",
                })
        else:
          table.append(
                {
                    "File Name": file.name,
                    "Name": name,
                    "Cod": cod,
                    "HTML": "-",
                    "Doc": "Общий анализ крови_экспресс",
                    "Find": "Нет",
                }
                )

    table_df = pd.DataFrame( table )
    table_df.to_excel(path_GeneralBloodAnalysis,engine='xlsxwriter')
    print( "Загрузка документов вида \"Общий анализ крови\" и \"Общий анализ крови-экспрес\" окончена, данные выгружены в таблицу : \n\t\t"+path_GeneralBloodAnalysis )
    return table_df

def get_GeneralBloodAnalysis_doc(text,find_string,doc):

    res = [  ]

    while ( text.find(find_string) != -1  ):
      index_beging = text.find(find_string)
      if( index_beging == -1 ):
        return None
      text = text[index_beging:]

      iter_line = re.finditer("<hr style=\"color:green;\">",text)
      index_line = []
      for i in iter_line:
        index_line.append( i.start() )

      if( len(index_line)>0 ):

        html = text[:index_line[0]]

        soup = BeautifulSoup(html, 'lxml')
        name = soup.find('i')
        if( name != None ):
          name = name.text
          fio = re.findall( "[А-ЯЁA-Z]", name )
          if( len(fio) == 2 ):
            name = name.split(" ")
            name = name[0]+" "+fio[1]
          elif( len(fio) == 3 ):
            name = name.split(" ")
            name = name[0]+" "+fio[1]+fio[2]
          else:
            name = "-"

        cod = soup.find('b',string = re.compile( "Медицинская карта №" ) )
        if( cod != None ):
          cod = cod.find_next( "i" ).text
          cod = re.findall( "\d+", cod )[0]
        else:
          cod = "-"
        res.append(
                    {
                      "Name":name,
                      "Cod":cod,
                      "HTML":html,
                      "Doc":doc
                    }
             )
        text = text[index_line[0]:]
      else:
        res.append(
                    {
                      "Name":"-",
                      "Cod":"-",
                      "HTML":"Конец документа не обнаружен",
                      "Doc":doc
                    }
             )
        break

    return  res

#------------------------------------------------------------------------------------Выгрузка документов вида "Общий анализ крови" и "Общий анализ крови-экспрес"


def get_GeneralBloodAnalysis_values( GeneralBloodAnalysis ):

    GeneralBloodAnalysis["Время выполнения"] = "-"
    GeneralBloodAnalysis["Дата направления"] = "-"
    GeneralBloodAnalysis["Дата взятия биоматериала"] = "-"

    for index in tqdm(GeneralBloodAnalysis.index):

      name_str = GeneralBloodAnalysis["File Name"][index].split( "/" )[-1]
      name_str = name_str.split( "_" )[0]
      cod =  re.findall( "\d+", name_str )[0]
      name_str = name_str[ : len(name_str) - len(cod) ]
      name = re.findall("[А-ЯЁAZ][А-ЯЁAZ]",name_str)
      if( len(name) > 0  ):
        name = name_str.replace( name[0], " "+name[0] )
      else:
        name = name_str[:-2]+" "+name_str[-2:]
      print(name,cod)

      GeneralBloodAnalysis["Cod"][index] = cod
      GeneralBloodAnalysis["Name"][index] = name

      if( GeneralBloodAnalysis[ "Find" ][index] == "Да" ):
          #print( GeneralBloodAnalysis["HTML"][index] )
          html = GeneralBloodAnalysis["HTML"][index]
          soup = BeautifulSoup(html, 'lxml')

          time_lead  = re.findall( "Выполнен \d+\.\d+\.\d+ в \d+:\d+", html )
          if( len(time_lead)>0 ):
              time_lead  = re.findall( "\d+\.\d+\.\d+ в \d+:\d+", time_lead[0] )
              try:
                time_lead = datetime.datetime.strptime( time_lead[0], '%d.%m.%Y в %H:%M' )
              except ValueError as e:
                time_lead = e
                print(time_lead)
          else:
            time_lead = "-"

          time_referral = soup.find( "b", string = re.compile( "Дата направления" ) )
          if( time_referral != None ):
            time_referral = time_referral.find_next( "i" )
            if( time_referral ):
              try:
                time_referral = datetime.datetime.strptime( time_referral.text, '%d.%m.%Y в %H:%M' )
              except ValueError as e:
                time_referral = e
                print(time_referral)
            else:
              time_referral = "-"
          else:
            time_referral = "-"

          date_sampling = soup.find( "b", string = re.compile( "Дата взятия биоматериала" ) )
          if( date_sampling != None ):
            date_sampling = date_sampling.find_next( "i" )
            if( date_sampling ):
              date_sampling = date_sampling.text
            else:
              date_sampling = None
          else:
            date_sampling = None

          time_sampling = soup.find( "b", string = re.compile( "Время" ) )
          if( time_sampling != None ):
            time_sampling = time_sampling.find_next( "i" )
            if( time_sampling ):
              time_sampling = time_sampling.text
            else:
              time_sampling = None
          else:
            time_sampling = None


          if( date_sampling != None and time_sampling != None ):
            try:
                time_sampling = datetime.datetime.strptime( date_sampling+" "+time_sampling, '%d.%m.%Y %H:%M' )
            except ValueError as e:
                time_sampling = e
                print(time_sampling)
          else:
            time_sampling = "-"

          GeneralBloodAnalysis["Время выполнения"][index] = time_lead
          GeneralBloodAnalysis["Дата направления"][index] = time_referral
          GeneralBloodAnalysis["Дата взятия биоматериала"][index] = time_sampling

    GeneralBloodAnalysis.to_excel(path_GeneralBloodAnalysis,engine='xlsxwriter')


def get_GeneralBloodAnalysis(  ):
    GeneralBloodAnalysis_df = get_GeneralBloodAnalysis_docs()
    get_GeneralBloodAnalysis_values( GeneralBloodAnalysis_df )


In [7]:
df = get_GeneralBloodAnalysis()
#df.to_excel()

Загрузка документов вида "Общий анализ крови" и "Общий анализ крови-экспрес" начата


100%|██████████| 171/171 [00:06<00:00, 24.93it/s]


Загрузка документов вида "Общий анализ крови" и "Общий анализ крови-экспрес" окончена, данные выгружены в таблицу : 
		/content/drive/MyDrive/Магистратура/Научная работа/DataSetes/Histores/Docs/Общий анализ крови Онкалогия.xlsx


  6%|▌         | 32/562 [00:00<00:01, 313.86it/s]

Лепейко ГВ 4787
Лепейко ГВ 4787
Белкин СВ 6996
Белкин СВ 6996
Рожков ВА 11200
Рожков ВА 11200
Романов АГ 11685
Романов АГ 11685
Ковзун СИ 3155
Ковзун СИ 3155
Комендантова ЛФ 8578
Комендантова ЛФ 8578
Коробко НВ 8822
Коробко НВ 8822
Ерина МС 9354
Ерина МС 9354
Дехтеренко СЕ 11699
Дехтеренко СЕ 11699
Духанин ВФ 13905
Духанин ВФ 13905
Ковпак ТГ 1894
Ковпак ТГ 1894
Стреха РИ 1669
Стреха РИ 1669
Рыбникова ЛН 5834
Рыбникова ЛН 5834
Изофатов АВ 6655
Изофатов АВ 6655
Лебедева АД 10784
Лебедева АД 10784
Лебедева АД 10784
Лебедева АД 10784
Сысик ВИ 11238
Сысик ВИ 11238
Сысик ВИ 11238
Сысик ВИ 11238
Пивенко ВЕ 12290
Пивенко ВЕ 12290
Пивенко ВЕ 12290
Пивенко ВЕ 12290
Долотов СА 12688
Долотов СА 12688
Долотов СА 12688
Глазкова ОП 15136
Глазкова ОП 15136
Глазкова ОП 15136
Глазкова ОП 15136
Лобода ВГ 16226
Лобода ВГ 16226
Лобода ВГ 16226
Лобода ВГ 16226
Лобода ВГ 16226
Лобода ВГ 16226
Лобода ВГ 16226
Малышкин ВС 381
Малышкин ВС 381
Малышкин ВС 381
Варнавский ГС 4757
Варнавский ГС 4757
Варнавский

...

Туркутюков ВБ 13064
Туркутюков ВБ 13064
Туркутюков ВБ 13064
Туркутюков ВБ 13064
Маринцев ВИ 12423
Маринцев ВИ 12423
Краснолобова НН 14299
Краснолобова НН 14299
Краснолобова НН 14299
Краснолобова НН 14299
Краснолобова НН 14299
Краснолобова НН 14299
Краснолобова НН 14299
Краснолобова НН 14299
Сапунов ВФ 12799
Сапунов ВФ 12799
Сапунов ВФ 12799
Рощупкин СА 13301


 97%|█████████▋| 543/562 [00:05<00:00, 112.79it/s]

Рощупкин СА 13301
Коротких ИВ 8683
Коротких ИВ 8683
Коротких ИВ 8683
Засемкова ОН 13588
Засемкова ОН 13588
Засемкова ОН 13588
Засемкова ОН 13588
Засемкова ОН 13588
Засемкова ОН 13588
Холопова ЛП 13897
Холопова ЛП 13897
Слюсарев АГ 11103
Слюсарев АГ 11103
Киняйкин МИ 11198
Киняйкин МИ 11198
Иваник ЛК 11765
Иваник ЛК 11765
Патуринский ВВ 15264
Патуринский ВВ 15264
Патуринский ВВ 15264
Патуринский ВВ 15264
Патуринский ВВ 15264
Житяйкин ИБ 15868
Житяйкин ИБ 15868
Пищулина МЕ 19392


100%|██████████| 562/562 [00:05<00:00, 93.90it/s] 


Пищулина МЕ 19392
Ткачук ГД 16809
Ткачук ГД 16809
Медвецкая ЛА 20487
Медвецкая ЛА 20487
Кошмин ЕА 20732
Кошмин ЕА 20732
Кошмин ЕА 20732
Рудницкий ГА 18258
Рудницкий ГА 18258


In [ ]:
path_DataSet = r'/content/drive/MyDrive/Магистратура/Научная работа/DataSetes/DataSet_V47.xlsx'

data = pd.read_excel( path_DataSet )

data["Смерть дих"] = 0
data.loc[((data["Смерть"])=="Да") ,('Смерть дих')]=1
data.loc[((data["Смерть"])==True) ,('Смерть дих')]=1
data["Смерть_Категория"] = "Выживший"
data.loc[((data["Смерть"])=="Да"),('Смерть_Категория')]="Умерший"
data.loc[((data["Смерть"] == True)),('Смерть_Категория')]="Умерший"

data = data.loc[ (data["Наличие в файле"] == "Да") | (data["Наличие в файле"] == True)]
data = data.loc[ (data["Наличие в БД"] == "Да") | (data["Наличие в БД"] == True)]
data = data.loc[ (data["STEMI"] == "Да") | (data["STEMI"] == True) ]
data = data.loc[ (data["ЧКВ"] == "Да") | (data["ЧКВ"] == True) ]

data = data.loc[ (data["Смерть дих"] == 1) ]
print(data)

In [4]:
import sys
path_source_cod = r'/content/drive/MyDrive/Магистратура/Научная_работа/Parser/Source/'
sys.path.append(path_source_cod)

from constants import path_PrimaryMedicalExamination, path_HTML_history
from get_name import get_name_history
from get_cod import get_cod_history

import re
import datetime
from os import walk, listdir
from bs4 import BeautifulSoup
from tqdm import tqdm
import datetime
import pandas as pd
import numpy as np

#------------------------------------------------------------------------------------Выгрузка документов вида "Первичный врачебный осмотр"

def get_PrimaryMedicalExamination_docs():
    table = []
    #z = 0
    print( "Загрузка документов вида \"Первичный врачебный осмотр\" начата" )
    for file in tqdm(listdir(path_HTML_history)):
        if '.html' not in file:
                continue
        with open(f'{path_HTML_history}/{file}', encoding='windows-1251') as file:
                        src = file.read()
        soup = BeautifulSoup(src, 'lxml')
        name = None
        cod = None
        find = "Нет"

        name = get_name_history(soup)
        cod = get_cod_history(soup)

        htmls = get_PrimaryMedicalExamination_doc(src)
        if( len(htmls) > 0 and htmls[0] != None ):
          find = "Да"
          for html in htmls :
            table.append(
                {
                    "File Name": file.name,
                    "Name": str(name),
                    "Cod": str(cod),
                    "Find": find,
                    "HTML": html,
                }
          )
        else:
          table.append(
                {
                    "File Name": file.name,
                    "Name": str(name),
                    "Cod": str(cod),
                    "Find": find,
                    "HTML": None,
                }

        )
        #z = z +1
        #if( z > 100 ):
        #  break

    table_df = pd.DataFrame( table )
    table_df.to_excel(path_PrimaryMedicalExamination,engine='xlsxwriter')
    print( "Загрузка документов вида \"Первичный врачебный осмотр\" окончена, данные выгружены в таблицу : \n\t\t"+path_PrimaryMedicalExamination )
    return table_df

def get_PrimaryMedicalExamination_doc(text):
    text_reserv = text
    res = [  ]
    doc = "<h4>Первичный врачебный осмотр</h4>"

    while ( text.find(doc) != -1  ):
      index_beging = text.find(doc)
      if( index_beging == -1 ):
        return None

      text = text[index_beging:]

      iter_line = re.finditer("<hr>",text)
      index_line = []
      for i in iter_line:
        index_line.append( i.start() )

      if( len(index_line)>=2 ):
        res.append( text[:index_line[1]] )
        text = text[index_line[1]:]
      else:
        res.append( None )
        text = text[index_line[0]:]

    return res

def get_PrimaryMedicalExamination_value( PrimaryMedicalExamination ):

    print( "Разбор документов вида \"Первичный врачебный осмотр\" начат" )
    test_df = pd.DataFrame()
    test_df["File"] = None
    test_df["Name"] = None
    test_df["Cod"] = None
    test_df["HTML"] = None
    test_df["Дата осмотра"] = None
    test_df["Инфаркт"] = None
    test_df["Инфаркт строкой"] = None
    test_df["Дата инфаркта"] = None
    test_df["Инфаркт в анамнезе <3"] = None
    test_df["Инфаркт в анамнезе >3"] = None
    test_df["Стентирование и инфаркт"] = None


    z = 0


    PrimaryMedicalExamination["Возраст"] = None
    PrimaryMedicalExamination["Ишемический инсульт"] = None
    PrimaryMedicalExamination["Геморрагический инсульт"] = None
    PrimaryMedicalExamination["Инсульт"] = None
    PrimaryMedicalExamination["Инсульт отрицает"] = None
    PrimaryMedicalExamination["ХОБЛ"] = None
    PrimaryMedicalExamination["Курение"] = None
    PrimaryMedicalExamination["Курение строка"] = None
    PrimaryMedicalExamination["Длительность курения"] = None
    PrimaryMedicalExamination["Количество сигарет в день"] = None
    PrimaryMedicalExamination["Количество пачек в день"] = None
    PrimaryMedicalExamination["Индекс курения"] = None
    PrimaryMedicalExamination["Отрицание курения"] = None
    PrimaryMedicalExamination["Инфаркт миокарда старый"] = None
    PrimaryMedicalExamination["Инфаркт миокарда отрицает"] = None
    PrimaryMedicalExamination["Дата осмотра"] = None
    PrimaryMedicalExamination["Отек легких"] = None
    PrimaryMedicalExamination["Отек легких строка"] = None
    PrimaryMedicalExamination["ОСН по Killip строка"] = None
    PrimaryMedicalExamination["ОСН по Killip"] = None

    for index in tqdm(PrimaryMedicalExamination.index):
        html = PrimaryMedicalExamination["HTML"][index]
        if( type(html) == float or html == None ):
            continue

        html = html.replace("&nbsp;"," ")
        hobl = None
        #---------------------------------------------------------------------------Возраст
        age = re.findall( "Возраст <i>\d\d|Возраст <i>\d\d", html)
        if( len(age)>0 ):
          age = re.findall( "\d\d", age[0])
          age = int(age[0])
        else:
          age = "-"

        #---------------------------------------------------------------------------ХОБЛ
        if( re.search( "Диагноз:", html) and (re.search( "План ведения и лечения:", html) or re.search( "Врач:", html)) ):

            beging = html.find( "Диагноз:" )
            end = html.find( "План ведения и лечения:" )
            if( end == -1 ):
              end = html.find( "Врач:" )
            hobl = html[ beging:end ]
            hobl = hobl.lower()
            if( re.search("хобл",hobl)  ):
              hobl = "Да"
            else:
              hobl = "Нет"
        else:
          hobl = "-"

        #---------------------------------------------------------------------------Инсульт
        isIsh = re.search( "онмк[ по]*ишемич[а-яе \-\(\)]*|ишемич[а-яе \-\(\)]*инсульт",html )
        isGem = re.search( "онмк[ по]*геморрагич[а-яе \-\(\)]*|геморраги[а-яе \-\(\)]*инсул|геморраческого инсульта",html )
        noInsult = re.search( "инсульт[а-яе ,]*отрица|средний брат-инсульт|инсульты и инфаркты у отца|отца-гб,им,умер от инсульта|смерть отца в 44 года от инсульта|инсульт в анамнезе: нет|мать умерла от инсульта в 70 лет|отца[а-яе ,-]*инсульт|риск инсульта|страх перенести инсульт|инсульт,гб у матери|умер в54г от инсульта|у матери аг, инсульт|умерла от инсульта|инсульты со стороны матери|инсультов ранее не было|инсульт у брата|инсульт у матери|инсульт у отца|инсульты у отца|матери - инсульт|умерла от инсульта|расценил как инсульт|отец умер от инсульта|в78лет от инсульта|данных за инсульт нет|риск инсульта|умерла в 62г от инсульта|умерла в 88лет от инсульта|наследственность[а-яе :,\-\(\)]*онмк|наследственность[а-яе :,\-\(\)]*инсульт|онмк[а-яе :,\-\(\)]*отриц|онмк[а-яе :,\-\(\)]*отр\.|онмк[а-яе :,\-\(\)]*нет",html )
        isInsult = re.search( " инсульт|онмк",html )
        if( isIsh ):
            isIsh = "Да"
        else:
            isIsh = "Нет"

        if( isGem ):
            isGem = "Да"
        else:
            isGem = "Нет"

        if( noInsult ):
            noInsult = "Да"
        else:
            noInsult = "Нет"

        if( isInsult and noInsult == "Нет" ):
            isInsult = "Да"
        else:
            isInsult = "Нет"

        #---------------------------------------------------------------------------Курение
        html = html.lower()
        no_smoking = re.search( "не[ ]*кур|курил",html )
        smoking = re.findall( "кури",html )
        if( not no_smoking and len(smoking)>0 ):
          smoking_index = html.find(smoking[0])
          smoking_str = html[smoking_index-100:smoking_index+100]
          smoking = "Да"

          smoking_str = smoking_str.replace("год","лет")
          smoking_time = re.findall( "кур[,а-яе \d]*с[ ]*\d\d[ ]*лет",smoking_str )
          if( len(smoking_time)>0 ):
            smoking_time = re.findall("\d\d",smoking_time[0])
            smoking_time = age - int(smoking_time[0])
          else:
            smoking_time = re.findall( "кур[,а-яе \d]*\d+[ ]*лет",smoking_str )
            if( len(smoking_time) ):
              smoking_time = re.findall("\d+[ ]*лет",smoking_time[0])
              smoking_time = re.findall("\d+",smoking_time[0])
              smoking_time = int(smoking_time[0])
            else:
              smoking_time = "-"

          smoking_sig = re.findall( "\d+[ ]*сиг|\d+-\d+[ ]*сиг",smoking_str )
          if( len(smoking_sig) ):
            smoking_sig = re.findall("\d+-\d+",smoking_sig[0])
            if( len(smoking_sig)>0  ):
              smoking_sig = re.findall("\d+",smoking_sig[0])
              smoking_sig = (int(smoking_sig[1])+int(smoking_sig[0]))/2
            else:
              smoking_sig = re.findall( "\d+[ ]*сиг",smoking_str )
              smoking_sig = re.findall("\d+",smoking_sig[0])
              smoking_sig = int(smoking_sig[0])

            smoking_pach = smoking_sig/20
          else:
            smoking_sig = "-"
            smoking_str_tmp = smoking_str.replace( "п\день","пачек в день")
            smoking_str_tmp = smoking_str_tmp.replace( "п/день","пачек в день")
            smoking_str_tmp = smoking_str_tmp.replace( "п в день","пачек в день")
            smoking_str_tmp = smoking_str_tmp.replace( "пачке в день","пачек в день")
            smoking_str_tmp = smoking_str_tmp.replace( "пачку в день","пачек в день")
            smoking_str_tmp = smoking_str_tmp.replace( "пачки в день","пачек в день")
            smoking_str_tmp = smoking_str_tmp.replace( "пачка в день","пачек в день")
            smoking_str_tmp = smoking_str_tmp.replace( "п/д","пачек в день")
            smoking_str_tmp = smoking_str_tmp.replace( "п\д","пачек в день")
            smoking_str_tmp = smoking_str_tmp.replace( "до пачки сигарет в сутки","пачек в день")
            smoking_str_tmp = smoking_str_tmp.replace( "пачек сигарет","пачек в день")
            smoking_str_tmp = smoking_str_tmp.replace( "пачке сигарет в день","пачек в день")
            smoking_str_tmp = smoking_str_tmp.replace( "пачке","пачек в день")
            smoking_str_tmp = smoking_str_tmp.replace( "п\с","пачек в день")
            smoking_str_tmp = smoking_str_tmp.replace( "пачка","пачек в день")
            smoking_str_tmp = smoking_str_tmp.replace( "\\","/")

            if(re.search( "\d-\d[ ]*пачек в день",smoking_str_tmp )):
              smoking_pach = re.findall( "\d-\d[ ]*пачек в день",smoking_str_tmp )
              smoking_pach = re.findall( "\d",smoking_pach[0])
              smoking_pach = (int(smoking_pach[0])+int(smoking_pach[1]))/2
            elif(re.search( "\d\.\d[ ]*пачек в день",smoking_str_tmp )):
              smoking_pach = re.findall( "\d\.\d[ ]*пачек в день",smoking_str_tmp )
              smoking_pach = re.findall( "\d\.\d",smoking_pach[0] )
              smoking_pach = float(smoking_pach[0])
            elif(re.search( "\d,\d[ ]*пачек в день",smoking_str_tmp )):
              smoking_pach = re.findall( "\d\,\d[ ]*пачек в день",smoking_str_tmp )
              smoking_pach = re.findall( "\d\,\d",smoking_pach[0] )
              smoking_pach = smoking_pach[0].replace(",",".")
              smoking_pach = float(smoking_pach)
            elif(re.search( "\d\\\d[ ]*пачек в день",smoking_str_tmp )):
              smoking_pach = re.findall( "\d\\\d[ ]*пачек в день",smoking_str_tmp )
              smoking_pach = re.findall( "\d",smoking_pach[0])
              smoking_pach = (int(smoking_pach[0])/int(smoking_pach[1]))
            elif(re.search( "\d\/\d[ ]*пачек в день",smoking_str_tmp )):
              smoking_pach = re.findall( "\d\/\d[ ]*пачек в день",smoking_str_tmp )
              smoking_pach = re.findall( "\d",smoking_pach[0])
              smoking_pach = (int(smoking_pach[0])/int(smoking_pach[1]))
            elif(re.search( "\d[ ]*пачек в день",smoking_str_tmp )):
              smoking_pach = re.findall( "\d[ ]*пачек в день",smoking_str_tmp )
              smoking_pach = re.findall( "\d",smoking_pach[0])
              smoking_pach = int(smoking_pach[0])
            elif(re.search( "пачек в день",smoking_str_tmp )):
              smoking_pach = 1
            else:
              smoking_pach = "-"

        elif( no_smoking ):

          smoking = re.findall( "кури",html )
          if( len(smoking)==0 ):
            continue
          smoking_index = html.find(smoking[0])
          smoking_str = html[smoking_index-100:smoking_index+100]

          smoking_str_tmp = smoking_str
          smoking_str_tmp = smoking_str_tmp.replace("не курит","")

          if( re.search( "кури|ранее",smoking_str_tmp ) ):
            smoking = "Да"
          else:
            smoking = "Нет"

          smoking_str = smoking_str.replace("год","лет")
          smoking_time = re.findall( "кур[илт]+[ ]*с[ ]*\d+[ ]*до[ ]*\d+",smoking_str_tmp )
          if( len(smoking_time)>0 ):
            smoking_time = re.findall("\d\d",smoking_time[0])
            smoking_time = int(smoking_time[1]) - int(smoking_time[0])
          else:
            smoking_time = re.findall( "кур[илт]+[ ]*с[ ]*\d+[ ]*лет",smoking_str_tmp )
            if( len(smoking_time)>0 ):
              smoking_time = re.findall("\d+",smoking_time[0])
              smoking_time = age - int(smoking_time[0])
            else:
              smoking_time = re.findall( "кур[,а-яе ]*\d+[ ]*лет|ранее[ ]*\d+[ ]*лет",smoking_str_tmp )
              if( len(smoking_time) ):
                smoking_time = re.findall("\d+[ ]*лет",smoking_time[0])
                smoking_time = re.findall("\d+",smoking_time[0])
                smoking_time = int(smoking_time[0])
              else:
                smoking_time = "-"

          smoking_sig = re.findall( "\d+[ ]*сиг|\d+-\d+[ ]*сиг",smoking_str )
          if( len(smoking_sig) ):
            smoking_sig = re.findall("\d+-\d+",smoking_sig[0])
            if( len(smoking_sig)>0  ):
              smoking_sig = re.findall("\d+",smoking_sig[0])
              smoking_sig = (int(smoking_sig[1])+int(smoking_sig[0]))/2
            else:
              smoking_sig = re.findall( "\d+[ ]*сиг",smoking_str )
              smoking_sig = re.findall("\d+",smoking_sig[0])
              smoking_sig = int(smoking_sig[0])

            smoking_pach = smoking_sig/20
          else:
            smoking_sig = "-"
            smoking_str_tmp = smoking_str.replace( "п\день","пачек в день")
            smoking_str_tmp = smoking_str_tmp.replace( "п/день","пачек в день")
            smoking_str_tmp = smoking_str_tmp.replace( "п в день","пачек в день")
            smoking_str_tmp = smoking_str_tmp.replace( "пачке в день","пачек в день")
            smoking_str_tmp = smoking_str_tmp.replace( "пачку в день","пачек в день")
            smoking_str_tmp = smoking_str_tmp.replace( "пачки в день","пачек в день")
            smoking_str_tmp = smoking_str_tmp.replace( "пачка в день","пачек в день")
            smoking_str_tmp = smoking_str_tmp.replace( "п/д","пачек в день")
            smoking_str_tmp = smoking_str_tmp.replace( "п\д","пачек в день")
            smoking_str_tmp = smoking_str_tmp.replace( "до пачки сигарет в сутки","пачек в день")
            smoking_str_tmp = smoking_str_tmp.replace( "пачке сигарет в день","пачек в день")
            smoking_str_tmp = smoking_str_tmp.replace( "пачек сигарет","пачек в день")
            smoking_str_tmp = smoking_str_tmp.replace( "пачке","пачек в день")
            smoking_str_tmp = smoking_str_tmp.replace( "п\с","пачек в день")
            smoking_str_tmp = smoking_str_tmp.replace( "пачка","пачек в день")
            smoking_str_tmp = smoking_str_tmp.replace( "\\","/")

            if(re.search( "\d-\d[ ]*пачек в день",smoking_str_tmp )):
              smoking_pach = re.findall( "\d-\d[ ]*пачек в день",smoking_str_tmp )
              smoking_pach = re.findall( "\d",smoking_pach[0])
              smoking_pach = (int(smoking_pach[0])+int(smoking_pach[1]))/2
            elif(re.search( "\d\.\d[ ]*пачек в день",smoking_str_tmp )):
              smoking_pach = re.findall( "\d\.\d[ ]*пачек в день",smoking_str_tmp )
              smoking_pach = re.findall( "\d\.\d",smoking_pach[0] )
              smoking_pach = float(smoking_pach[0])
            elif(re.search( "\d,\d[ ]*пачек в день",smoking_str_tmp )):
              smoking_pach = re.findall( "\d\,\d[ ]*пачек в день",smoking_str_tmp )
              smoking_pach = re.findall( "\d\,\d",smoking_pach[0] )
              smoking_pach = smoking_pach[0].replace(",",".")
              smoking_pach = float(smoking_pach)
            elif(re.search( "\d\\\d[ ]*пачек в день",smoking_str_tmp )):
              smoking_pach = re.findall( "\d\\\d[ ]*пачек в день",smoking_str_tmp )
              smoking_pach = re.findall( "\d",smoking_pach[0])
              smoking_pach = (int(smoking_pach[0])/int(smoking_pach[1]))
            elif(re.search( "\d\/\d[ ]*пачек в день",smoking_str_tmp )):
              smoking_pach = re.findall( "\d\/\d[ ]*пачек в день",smoking_str_tmp )
              smoking_pach = re.findall( "\d",smoking_pach[0])
              smoking_pach = (int(smoking_pach[0])/int(smoking_pach[1]))
            elif(re.search( "\d[ ]*пачек в день",smoking_str_tmp )):
              smoking_pach = re.findall( "\d[ ]*пачек в день",smoking_str_tmp )
              smoking_pach = re.findall( "\d",smoking_pach[0])
              smoking_pach = int(smoking_pach[0])
            elif(re.search( "пачек в день",smoking_str_tmp )):
              smoking_pach = 1
            else:
              smoking_pach = "-"
        else:
          smoking_str = "-"
          smoking = "Нет"
          smoking_time = "-"
          smoking_sig = "-"
          smoking_pach = "-"

        if( smoking_time != "-" and smoking_pach != "-" ):
          smoking_index = smoking_pach*smoking_time
        else:
          smoking_index = "-"

        #---------------------------------------------------------------------------Дата осмотра
        html = PrimaryMedicalExamination["HTML"][index]
        html = html.replace("&nbsp;"," ")
        data_osmotr = re.findall( "Дата и время осмотра <i>\d\d.\d\d.\d\d\d\d",html )
        if( len(data_osmotr)>0 ):
          data_osmotr = re.findall( "\d\d.\d\d.\d\d\d\d",data_osmotr[0] )
          data_osmotr = data_osmotr[0]
          data_osmotr = datetime.datetime.strptime(data_osmotr, '%d.%m.%Y')
        else:
          data_osmotr = "-"

        #---------------------------------------------------------------------------Инфаркт
        html = html.lower()
        #infarcts_old = re.search( "пикс|постинфарктн",html )
        infarcts_otric = re.search( "[ о,\.]им[а-яё, a-z]*отриц|[ о,\.]им[а-яё, a-z]*нет|[ о,\.]им[а-яё, a-z]*не да[её]т|[ о,\.]инфаркт[а-яё, a-z]*отриц|[ о,\.]инфаркт[а-яё, a-z]*нет|[ о,\.]инфаркт[а-яё, a-z]*не да[её]т|ибс[а-яё, a-z]*отриц|ибс[а-яё, a-z]*нет|ибс[а-яё, a-z]*не да[её]т",html )

        name_file = PrimaryMedicalExamination["File Name"][index]
        name = PrimaryMedicalExamination["Name"][index]
        cod = PrimaryMedicalExamination["Cod"][index]


        for infarct in re.finditer("[ о,\.()]им[ пs]|пикс|инфаркт",html):
          infarct_str = html[infarct.start()-20:infarct.start()+120]

          try:
            infarct_str_data = re.findall( "[ о,\.\d]им[а-яё,\. a-z\-]*\d\d.\d\d.\d\d\d\d|пикс[а-яё,\. a-z\-]*\d\d.\d\d.\d\d\d\d|инфаркт[а-яё,\. a-z\-]*\d\d.\d\d.\d\d\d\d",infarct_str)
            if( len(infarct_str_data)>0 ):
              infarct_str_data = re.findall("\d\d.\d\d.\d\d\d\d",infarct_str_data[0])[0]
              infarct_str_data = infarct_str_data.replace( ",", "." )
              infarct_str_data = infarct_str_data.replace( "\\", "." )
              infarct_str_data = infarct_str_data.replace( "/", "." )
              infarct_str_data = infarct_str_data.replace( "ю", "." )
              infarct_str_data = infarct_str_data.replace( "\\\\", "." )
              infarct_str_data = datetime.datetime.strptime(infarct_str_data, '%d.%m.%Y')

            elif( len(re.findall( "[ о,\.]им[а-яё,\. a-z\-]*\d\d.\d\d.\d\d|пикс[а-яё,\. a-z\-]*\d\d.\d\d.\d\d|инфаркт[а-яё,\. a-z\-]*\d\d.\d\d.\d\d",infarct_str))>0 ):
              infarct_str_data = re.findall( "[ о,\.]им[а-яё,\. a-z\-]*\d\d.\d\d.\d\d|пикс[а-яё,\. a-z\-]*\d\d.\d\d.\d\d|инфаркт[а-яё,\. a-z\-]*\d\d.\d\d.\d\d",infarct_str)
              infarct_str_data = re.findall("\d\d.\d\d.\d\d",infarct_str_data[0])[0]
              infarct_str_data = infarct_str_data.replace( ",", "." )
              infarct_str_data = infarct_str_data.replace( "\\", "." )
              infarct_str_data = infarct_str_data.replace( "/", "." )
              infarct_str_data = infarct_str_data.replace( "ю", "." )
              infarct_str_data = infarct_str_data.replace( "\\\\", "." )
              infarct_str_data = datetime.datetime.strptime(infarct_str_data, '%d.%m.%y')

            elif( re.search( "[ о,\.]им[а-яё, a-z()\-\.]*\d\d\d\d|пикс[а-яё, a-z()\-\.]*\d\d\d\d|инфаркт[а-яё, a-z()\-\.]*\d\d\d\d|пик[а-яё, a-z()\-\\\.]*\d\d\d\d|\d\d\d\d[а-яё ]*инфаркт",infarct_str) ):
              infarct_str_data = re.findall( "[ о,\.]им[а-яё, a-z()\-\.]*\d\d\d\d|пикс[а-яё, a-z()\-\.]*\d\d\d\d|инфаркт[а-яё, a-z()\-\.]*\d\d\d\d|пик[а-яё, a-z()\-\\\.]*\d\d\d\d|\d\d\d\d[а-яё ]*инфаркт",infarct_str)[0]
              infarct_str_data = re.findall("\d\d\d\d",infarct_str_data)[0]

              infarct_str_data = datetime.datetime.strptime( "01.06."+infarct_str_data, '%d.%m.%Y')

            elif( re.search( "[ о,\.]им[а-яё,\. a-z\-]*\d.\d\d.\d\d|пикс[а-яё,\. a-z\-]*\d.\d\d.\d\d|инфаркт[а-яё,\. a-z\-]*\d.\d\d.\d\d",infarct_str) ):
              infarct_str_data = re.findall( "[ о,\.]им[а-яё,\. a-z\-]*\d.\d\d.\d\d|пикс[а-яё,\. a-z\-]*\d.\d\d.\d\d|инфаркт[а-яё,\. a-z\-]*\d.\d\d.\d\d",infarct_str)[0]
              infarct_str_data = re.findall("\d.\d\d.\d\d",infarct_str_data)[0]
              infarct_str_data = infarct_str_data.replace( ",", "." )
              infarct_str_data = infarct_str_data.replace( "\\", "." )
              infarct_str_data = infarct_str_data.replace( "/", "." )
              infarct_str_data = infarct_str_data.replace( " ", "." )
              infarct_str_data = infarct_str_data.replace( "ю", "." )
              infarct_str_data = infarct_str_data.replace( "\\\\", "." )
              infarct_str_data = datetime.datetime.strptime( "0"+infarct_str_data, '%d.%m.%y')
            else:
              infarct_str_data = "-"
          except Exception as e:
            print( e )
            infarct_str_data = "-"

          if( infarct_str_data != "-" and data_osmotr != "-" ):

            ras = (data_osmotr - infarct_str_data).days
            if( ras>14 and ras<=90 ):
              dav_men_3 = "Да"
              dav_bol_3 = "Нет"
            elif( ras>90 ):
              dav_men_3 = "Нет"
              dav_bol_3 = "Да"
            else:
              dav_men_3 = "Нет"
              dav_bol_3 = "Нет"
          else:
              dav_men_3 = "Нет"
              dav_bol_3 = "Нет"

          if( re.search( "стентир|талбап",infarct_str ) ):
            stent = "Да"
          else:
            stent = "Нет"



          test_df.loc[ len(test_df.index )] = [name_file,
                                               name,
                                               cod,
                                               html,
                                               data_osmotr,
                                               "Да",
                                               infarct_str,
                                               infarct_str_data,
                                               dav_men_3,
                                               dav_bol_3,
                                               stent,
                                               ]



        #if( infarcts_old ):
        #  infarcts_old = "Да"
        #if( infarcts_otric ):
        #  infarcts_otric = "Да"
        #if( len(infarcts_old)>0 ):
        #  infarct_count = len(infarcts)
        #  infarct_str = ""
        #  for infarct in infarcts:
        #    infarct_index = html.find(infarct)
        #    infarct_str = infarct_str + html[infarct_index-60:infarct_index+60] + "\n"
        #else:
        #  infarct_str = "-"
        #  infarct_count = 0

        html = html.replace(">отек легких, обуслов","")
        html = html.replace(">отек легких, наиболе","")

        otec_leg_str = re.findall( "от[её]к[а ]*легких|интерстициальн[ыйого ]*от[её]к[а]*|интерстициальн[ыйого ]*от[её]к",html )
        otec_leg_str_no = re.findall( "отека легких нет|отека нет",html )
        otec_leg = "Нет"
        if( len(otec_leg_str) > 0 ):

          string_index = html.find(otec_leg_str[0])
          otec_leg_str = html[ string_index-20:string_index+20 ]

          if( len(otec_leg_str_no)>0 ):
            otec_leg = "Нет"
          else:
            otec_leg = "Да"
        else:
          otec_leg_str = "-"

        killip_str = re.findall( "killip|kilip",html )
        killip = "-"
        if( len(killip_str)>0 ):

          string_index = html.find(killip_str[0])
          killip_str = html[ string_index-20:string_index+20 ]

          killip_str = killip_str.replace( "iv","4" )

          killip_str = killip_str.replace( " ш "," 3 " )
          killip_str = killip_str.replace( " щ "," 3 " )
          killip_str = killip_str.replace( "iii","3" )
          killip_str = killip_str.replace( "lll","3" )
          killip_str = killip_str.replace( "lll","3" )

          killip_str = killip_str.replace( " п "," 2 " )
          killip_str = killip_str.replace( "ii"," 2 " )

          killip_str = killip_str.replace( "i ", "1 " )
          killip_str = killip_str.replace( " i"," 1" )
          killip_str = killip_str.replace( " i"," 1" )
          killip_str = killip_str.replace( "killipi","killip 1" )

          killip = re.findall( "killip[ -]*\d-\d|осн[ -]*\d-\d|\d-\d[ -]*killip",killip_str)
          if( len(killip)>0 ):
            killip = max([int( re.findall( "\d",killip[0] )[0] ),int( re.findall( "\d",killip[0] )[1] )])
          elif( killip == re.search( "killip[ -]*\d|осн[ -]*\d|\d[ -]*killip",killip_str) ):
            killip = re.findall( "killip[ -]*\d|осн[ -]*\d|\d[ -]*killip",killip_str)
            killip = int( re.findall( "\d",killip[0] )[0] )

        #PrimaryMedicalExamination["Ишемический инсульт"][index] = isIsh
        #PrimaryMedicalExamination["Геморрагический инсульт"][index] = isGem
        #PrimaryMedicalExamination["Инсульт"][index] = isInsult
        #PrimaryMedicalExamination["Инсульт отрицает"][index] = noInsult
        #PrimaryMedicalExamination["ХОБЛ"][index] = hobl
        #PrimaryMedicalExamination["Возраст"][index] = age
        #PrimaryMedicalExamination["Курение"][index] = smoking
        #PrimaryMedicalExamination["Отрицание курения"][index] = no_smoking
        #PrimaryMedicalExamination["Курение строка"][index] = smoking_str
        #PrimaryMedicalExamination["Длительность курения"][index] = smoking_time
        #PrimaryMedicalExamination["Количество сигарет в день"][index] = smoking_sig
        #PrimaryMedicalExamination["Количество пачек в день"][index] = smoking_pach
        #PrimaryMedicalExamination["Индекс курения"][index] = smoking_index
        #PrimaryMedicalExamination["Дата осмотра"][index] = data_osmotr
        #PrimaryMedicalExamination["Инфаркт миокарда старый"][index] = infarcts_old
        #PrimaryMedicalExamination["Инфаркт миокарда отрицает"][index] = infarcts_otric
        PrimaryMedicalExamination["Отек легких строка"][index] = otec_leg_str
        PrimaryMedicalExamination["Отек легких"][index] = otec_leg
        PrimaryMedicalExamination["ОСН по Killip строка"][index] = killip_str
        PrimaryMedicalExamination["ОСН по Killip"][index] = killip

        #if( z>2000 ):
        #  break
        #z=z+1


    PrimaryMedicalExamination.to_excel(path_PrimaryMedicalExamination+"_Тест.xlsx",engine='xlsxwriter')
    #PrimaryMedicalExamination.to_excel(path_PrimaryMedicalExamination+"_Тест.xlsx",engine='xlsxwriter')
    print( "Разбор документов вида \"Первичный врачебный осмотр\" окончен, данные сохранены в таблицу: \n\t\t"+path_PrimaryMedicalExamination )

def get_PrimaryMedicalExamination(  ):
    PrimaryMedicalExamination_df = get_PrimaryMedicalExamination_docs()
    get_PrimaryMedicalExamination_value( PrimaryMedicalExamination_df )

In [5]:
get_PrimaryMedicalExamination()

Загрузка документов вида "Первичный врачебный осмотр" начата


100%|██████████| 172/172 [00:44<00:00,  3.85it/s]


Загрузка документов вида "Первичный врачебный осмотр" окончена, данные выгружены в таблицу : 
		/content/drive/MyDrive/Магистратура/Научная работа/DataSetes/Histores/Docs/Первичный врачебный осмотр_Test.xlsx
Разбор документов вида "Первичный врачебный осмотр" начат


100%|██████████| 172/172 [00:00<00:00, 265.67it/s]


Разбор документов вида "Первичный врачебный осмотр" окончен, данные сохранены в таблицу: 
		/content/drive/MyDrive/Магистратура/Научная работа/DataSetes/Histores/Docs/Первичный врачебный осмотр_Test.xlsx


In [ ]:
PrimaryMedicalExamination = pd.read_excel(path_PrimaryMedicalExamination)

In [ ]:
get_PrimaryMedicalExamination_value( PrimaryMedicalExamination )
no_killip = PrimaryMedicalExamination.loc[PrimaryMedicalExamination["ОСН по Killip"]=="-"][["ОСН по Killip строка","ОСН по Killip"]]


In [ ]:
print(PrimaryMedicalExamination)
for index in no_killip.index:
  print( no_killip["ОСН по Killip строка"][index], no_killip["ОСН по Killip"][index] )

In [ ]:
print(PrimaryMedicalExamination.loc[PrimaryMedicalExamination["ОСН по Killip"]=="-"][["ОСН по Killip строка","ОСН по Killip"]])

                          ОСН по Killip строка ОСН по Killip
3                                           []             -
4                                           []             -
5                                           []             -
6                                           []             -
7    вного ds: <p>осн по killip i.</p>\n<p>хсн             -
..                                         ...           ...
498                                         []             -
499                                         []             -
501                                         []             -
505                                         []             -
506                                         []             -

[281 rows x 2 columns]


In [ ]:
print( r'pictures\12761_1.jpg'.replace("\\", "/"))

pictures/12761_1.jpg


In [ ]:
          age = 43
          smoking_str  = "ые продукты: нет. травмы: нет. операции: нет наследственность: нет. вредные привычки курит до 1 пачки в дент более 25 лет страхо"
          smoking_time = re.findall( "кур[,а-яе \d]*с[ ]*\d\d[ ]*лет",smoking_str )
          if( len(smoking_time)>0 ):
            smoking_time = re.findall("\d\d[ ]*лет",smoking_time[0])
            smoking_time = re.findall("\d\d",smoking_time[0])
            print(smoking_time)
            smoking_time = age - int(smoking_time[0])
          else:
            smoking_time = re.findall( "кур[,а-яе \d]*\d+[ ]*лет",smoking_str )
            if( len(smoking_time) ):
      smoking_time = re.findall("\d\d[ ]*лет",smoking_time[0])
              smoking_time = re.findall("\d\d",smoking_time[0])
              smoking_time = int(smoking_time[0])
            else:
              smoking_time = "-"
          print(smoking_time)

IndentationError: ignored

In [ ]:
import sys
path_source_cod = r'/content/drive/MyDrive/Магистратура/Научная_работа/Parser/Source/'
sys.path.append(path_source_cod)

from constants import path_ProceduralСardPatient, path_HTML_history
from get_name import get_name_history
from get_cod import get_cod_history

import re
from os import walk, listdir
from bs4 import BeautifulSoup
from tqdm import tqdm
import datetime
import pandas as pd
import numpy as np

#------------------------------------------------------------------------------------Выгрузка документов вида "Процедурная карта больного"

def get_ProceduralСardPatient_docs():
    table = []
    #z = 0
    print( "Загрузка документов вида \"Процедурная карта больного\" начата" )
    for file in tqdm(listdir(path_HTML_history)):
        if '.html' not in file:
                continue
        with open(f'{path_HTML_history}/{file}', encoding='windows-1251') as file:
                        src = file.read()
        soup = BeautifulSoup(src, 'lxml')
        name = None
        cod = None
        find = "Нет"

        name = get_name_history(soup)
        cod = get_cod_history(soup)

        htmls = get_ProceduralСardPatient_doc(src)
        if( len(htmls) > 0 and htmls[0] != None ):
          find = "Да"
          for html in htmls :
            table.append(
                {
                    "File Name": file.name,
                    "Name": str(name),
                    "Cod": str(cod),
                    "Find": find,
                    "HTML": html,
                }
          )
        else:
          table.append(
                {
                    "File Name": file.name,
                    "Name": str(name),
                    "Cod": str(cod),
                    "Find": find,
                    "HTML": None,
                }

        )
        #z = z +1
        #if( z > 100 ):
        #  break

    table_df = pd.DataFrame( table )
    table_df.to_excel(path_ProceduralСardPatient,engine='xlsxwriter')
    print( "Загрузка документов вида \"Процедурная карта больного\" окончена, данные выгружены в таблицу : \n\t\t"+path_ProceduralСardPatient )
    return table_df

def get_ProceduralСardPatient_doc(text):
    text_reserv = text
    res = [  ]
    doc = "<b>Процедурная карта больного</b>"

    while ( text.find(doc) != -1  ):
      index_beging = text.find(doc)
      if( index_beging == -1 ):
        return None

      text = text[index_beging:]

      iter_line = re.finditer("<hr>",text)
      index_line = []
      for i in iter_line:
        index_line.append( i.start() )

      if( len(index_line)>=0 ):
        res.append( text[:index_line[0]] )
        text = text[index_line[0]:]
      else:
        res.append( None )
        text = text[index_line[0]:]

    return res

def get_ProceduralСardPatient_value( ProceduralСardPatient ):

    print( "Разбор документов вида \"Процедурная карта больного\" начат" )

    ProceduralСardPatient["Отделение"] = None

    for index in tqdm(ProceduralСardPatient.index):
        html = ProceduralСardPatient["HTML"][index]
        if( type(html) == float or html == None ):
            continue

        html = html.replace("&nbsp;"," ")
        soup = BeautifulSoup(html, 'lxml')
        department = soup.find( "b", string = re.compile("Отделение"))

        #---------------------------------------------------------------------------Отделение

        if( department != None ):
            department = department.next_element.next_element
            department = department.text
        else:
            department = "-"

        ProceduralСardPatient["Отделение"][index] = department

    ProceduralСardPatient.to_excel(path_ProceduralСardPatient,engine='xlsxwriter')
    print( "Разбор документов вида \"Процедурная карта больного\" окончен, данные сохранены в таблицу: \n\t\t"+path_ProceduralСardPatient )


def get_ProceduralСardPatient(  ):
    ProceduralСardPatient_df = get_ProceduralСardPatient_docs()
    get_ProceduralСardPatient_value( ProceduralСardPatient )


In [ ]:
get_ProceduralСardPatient()

Загрузка документов вида "Процедурная карта больного" начата


 38%|███▊      | 65/171 [00:13<00:21,  4.94it/s]


KeyboardInterrupt: ignored

In [ ]:
ProceduralСardPatient = pd.read_excel(path_ProceduralСardPatient)

In [ ]:
get_ProceduralСardPatient_value( ProceduralСardPatient )

In [ ]:
ProceduralСardPatient

In [ ]:
import sys
path_source_cod = r'/content/drive/MyDrive/Магистратура/Научная_работа/Parser/Source/'
sys.path.append(path_source_cod)

from constants import path_Epicrisis, path_HTML_history
from get_name import get_name_history
from get_cod import get_cod_history

import re
from os import walk, listdir
from bs4 import BeautifulSoup
from tqdm import tqdm
import datetime
import pandas as pd
import numpy as np

#------------------------------------------------------------------------------------Выгрузка документов вида "Переводной эпикриз","Этапный эпикриз","Посмертный эпикриз","Посмертный эпикриз"

def get_Epicrisis_docs():
    table = []
    #z = 0
    print( "Загрузка документов вида \"Переводной эпикриз\",\"Этапный эпикриз\",\"Посмертный эпикриз\",\"Посмертный эпикриз\", начата" )
    for file in tqdm(listdir(path_HTML_history)):
        if '.html' not in file:
                continue
        with open(f'{path_HTML_history}/{file}', encoding='utf-8') as file:
                        src = file.read()
        soup = BeautifulSoup(src, 'lxml')
        name = None
        cod = None
        find = "Нет"

        name = get_name_history(soup)
        cod = get_cod_history(soup)

        htmls = get_Epicrisis_doc(src)
        if( len(htmls) > 0 and htmls[0] != None ):
          find = "Да"
          for html in htmls :
            table.append(
                {
                    "File Name": file.name,
                    "Name": str(name),
                    "Cod": str(cod),
                    "Find": find,
                    "Type": html["type_epicrisis"],
                    "HTML": html["html"],
                }
          )
        else:
          table.append(
                {
                    "File Name": file.name,
                    "Name": str(name),
                    "Cod": str(cod),
                    "Find": find,
                    "Type": None,
                    "HTML": None,
                }

        )
        #z = z +1
        #№if( z > 100 ):
        #  break

    table_df = pd.DataFrame( table )
    table_df.to_excel(path_Epicrisis,engine='xlsxwriter')
    print( "Загрузка документов вида \"Переводной эпикриз\",\"Этапный эпикриз\",\"Посмертный эпикриз\",\"Выписной эпикриз\" окончена, данные выгружены в таблицу : \n\t\t"+path_Epicrisis )
    return table_df

def get_Epicrisis_doc(text):
    text_reserv = text
    res = [  ]
    res = get_TransferableEpicrisis_doc(text,res)
    res = get_StageEpicrisis_doc(text,res)
    res = get_PosthumousEpicrisis_doc(text,res)
    res = get_DischargeEpicrisis_doc(text,res)

    return res

def get_TransferableEpicrisis_doc(text,res):
    text_reserv = text
    doc = ">Переводной эпикриз</"

    while ( text.find(doc) != -1  ):
      index_beging = text.find(doc)
      if( text.find(doc) == -1 ):
        return None

      text = text[index_beging:]
      iter_line = re.finditer("<hr>",text)
      index_line = []
      for i in iter_line:
        index_line.append( i.start() )

      if( len(index_line)>=4 ):
        res.append( {'html':text[:index_line[3]],'type_epicrisis':"Переводной эпикриз"} )
        text = text[index_line[3]:]
      else:
        res.append( None )
        text = text[index_line[0]:]

    return res

def get_StageEpicrisis_doc(text,res):
    text_reserv = text
    doc = "<b>Этапный эпикриз"
    while ( text.find(doc) != -1  ):
      index_beging = text.find(doc)
      if( index_beging == -1 ):
        return None

      text = text[index_beging:]

      iter_line = re.finditer("<hr>",text)
      index_line = []
      for i in iter_line:
        index_line.append( i.start() )

      if( len(index_line)>=3 ):
        res.append( {'html':text[:index_line[2]],'type_epicrisis':"Этапный эпикриз"} )
        text = text[index_line[2]:]
      else:
        res.append( None )
        text = text[index_line[0]:]

    return res

def get_PosthumousEpicrisis_doc(text,res):
    text_reserv = text
    doc = "<h4>Посмертный эпикриз</h4>"
    while ( text.find(doc) != -1  ):
      index_beging = text.find(doc)
      if( index_beging == -1 ):
        return None

      text = text[index_beging:]

      iter_line = re.finditer("<hr>",text)
      index_line = []
      for i in iter_line:
        index_line.append( i.start() )

      if( len(index_line)>=3 ):
        res.append( {'html':text[:index_line[2]],'type_epicrisis':"Посмертный эпикриз"} )
        text = text[index_line[2]:]
      else:
        res.append( None )
        text = text[index_line[0]:]

    return res

def get_DischargeEpicrisis_doc(text,res):
    text_reserv = text
    doc = ">В Ы П И С Н О Й &nbsp;&nbsp;&nbsp;&nbsp;  Э П И К Р И З<"
    while ( text.find(doc) != -1  ):
      index_beging = text.find(doc)
      if( index_beging == -1 ):
        return None

      text = text[index_beging:]

      iter_line = re.finditer("<hr>",text)
      index_line = []
      for i in iter_line:
        index_line.append( i.start() )

      if( len(index_line)>=3 ):
        #print("Find")
        res.append( {'html':text[:index_line[2]],'type_epicrisis':"Выписной эпикриз"} )
        text = text[index_line[2]:]
      else:
        res.append( None )
        text = text[index_line[0]:]

    return res

def get_Epicrisis_value( Epicrisis ):

    print( "Разбор документов вида \"Переводной эпикриз\",\"Этапный эпикриз\",\"Посмертный эпикриз\",\"Выписной эпикриз\" начат" )

    Epicrisis["ХОБЛ"] = None

    for index in tqdm(Epicrisis.index):
        html = Epicrisis["HTML"][index]
        if( type(html) == float ):
            continue

        #---------------------------------------------------------------------------ХОБЛ
        hobl = None
        if( html != None ):
          html = html.replace("&nbsp;"," ")
          html = html.lower()
          if( re.search( "хобл", html)):
            hobl = "Да"
          else:
            hobl = "Нет"
        else:
          hobl = "-"

        Epicrisis["ХОБЛ"][index] = hobl


    Epicrisis.to_excel(path_Epicrisis,engine='xlsxwriter')
    print( "Разбор документов вида \"Переводной эпикриз\",\"Этапный эпикриз\",\"Посмертный эпикриз\",\"Выписной эпикриз\" окончен, данные сохранены в таблицу: \n\t\t"+path_Epicrisis )


def get_Epicrisis(  ):
    Epicrisis_df = get_Epicrisis_docs()
    get_Epicrisis_value( Epicrisis_df )


In [ ]:
get_Epicrisis(  )

Загрузка документов вида "Переводной эпикриз","Этапный эпикриз","Посмертный эпикриз","Посмертный эпикриз", начата


100%|██████████| 15354/15354 [54:23<00:00,  4.71it/s]


Загрузка документов вида "Переводной эпикриз","Этапный эпикриз","Посмертный эпикриз","Выписной эпикриз" окончена, данные выгружены в таблицу : 
		/content/drive/MyDrive/Магистратура/Научная работа/DataSetes/Histores/Docs/Эпикризы.xlsx
Разбор документов вида "Переводной эпикриз","Этапный эпикриз","Посмертный эпикриз","Выписной эпикриз" начат


100%|██████████| 48962/48962 [00:18<00:00, 2651.87it/s]


Разбор документов вида "Переводной эпикриз","Этапный эпикриз","Посмертный эпикриз","Выписной эпикриз" окончен, данные сохранены в таблицу: 
		/content/drive/MyDrive/Магистратура/Научная работа/DataSetes/Histores/Docs/Эпикризы.xlsx


In [ ]:
get_Epicrisis_value( Epicrisis_df )

Разбор документов вида "Переводной эпикриз","Этапный эпикриз","Посмертный эпикриз","Выписной эпикриз" начат


  0%|          | 0/37286 [00:00<?, ?it/s]<ipython-input-21-7b8044a361e2>:199: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Epicrisis["ХОБЛ"][index] = hobl
100%|██████████| 37286/37286 [00:16<00:00, 2260.40it/s]


Разбор документов вида "Переводной эпикриз","Этапный эпикриз","Посмертный эпикриз","Выписной эпикриз" окончен, данные сохранены в таблицу: 
		/content/drive/MyDrive/Магистратура/Научная работа/DataSetes/Histores/Docs/Эпикризы.xlsx


In [ ]:
print(Epicrisis_df)



In [ ]:
names_file = """ММорозОИ10547_history_0.html
КрайневаЕИ_history_0.html
МоскаленкоАП_history_0.html
ПорхунАА13875_history_0.html
ЛесивВИ_history_0.html
ИвановаНН14239_history_0.html
ПрозороваММ_history_0.html
НасретдиновАГ_history_0.html
ОнанкоАИ_history_0.html
КарповЮЛ15868_history_0.html
ЩербаковАН10235_history_0.html
ШибановаЛН_history_0.html
ЛапшовАИ_history_0.html
ПозывайГА_history_0.html
ДацкоВВ_history_0.html
МоисеевВладимирАндреевич13370_history_0.html
СинельниковЮВ15273_history_0.html
ШуваеваНФ_history_0.html
СвидзинскаяВФ_history_0.html
ХакимовЮМ_history_0.html
АтабаевМУ_history_0.html
МоисееваЛП_history_0.html
ГайдамащукПП_history_0.html
ПервохинАБ_history_0.html
АльтерманАД_history_0.html
АбоносимовВИ_history_0.html
БендерскийНиколайРафаилович20840_history_0.html
СимоненкоВЯ_history_0.html
БуценкоЕГ_history_0.html
ЖарченкоНВ_history_0.html
ЩекалевКГ_history_0.html
МишинАФ_history_0.html
ДворянскаяТФ_history_0.html
ЛопатинНА7958_history_0.html
КузнецовАК_history_0.html
ГончаровИВ8706_history_0.html
АристоваНС_history_0.html
АртеменкоОА92_history_0.html
ЧелпановаРМ_history_0.html
БочкареваЕД_history_0.html
ПавловаОН_history_0.html
КудрявцеваКС_history_0.html
ЛукашовВД_history_0.html
ЧернояровБК_history_0.html
СкоринаЛН_history_0.html
МицСИ_history_0.html
ЛугинЕА_history_0.html
ХарченкоФН_history_0.html
ВерещагинаВА126_history_0.html
ЕлуферьеваТС_history_0.html
КлючникНЯ_history_0.html
СмирноваНФ13859_history_0.html
МакойдаАА_history_0.html
ШутьЛД_history_0.html
ЧурбаковаЛВ_history_0.html
КорховаТЗ_history_0.html
СазоновАлександрЕвгеньевич15728_history_0.html
-
ПедычЛД_history_0.html
НикулинВВ_history_0.html
КарбовскийФилиппАндреевич18966_history_0.html
ПетуховАВ19250_history_0.html
ДанковВадимАлександрович19878_history_0.html
МисюренкоСЯ_history_0.html
СавочкинаНЛ_history_0.html
МоскаленкоВА2135_history_0.html
ПодкорытоваВС_history_0.html
ПанинВИ3206_history_0.html
ПриемскаяТТ3319_history_0.html
ЛуканинВВ_history_0.html
ВертВР_history_0.html
ПоповаТС6095_history_0.html
ТретьяковаЛД_history_0.html
КузьменкоСН_history_0.html
ТаранНА8910_history_0.html
КизиловаТИ_history_0.html
КирьяновАВ9689_history_0.html
КирьяновАВ9689_history_0.html
ВласовСК_history_0.html
КостыринВФ_history_0.html
ЯрушинаТФ_history_0.html
МигашкоАВ_history_0.html
ГришинаНина11164_history_0.html
ДолгихВА_history_0.html
ИвановаАА13311_history_0.html
ЛопашукНИ_history_0.html
СапрыгинАА_history_0.html
МальцеваЛЯ_history_0.html
МоисееваГБ_history_0.html
ЗаварихинВВ14561_history_0.html
СлуженкоТА_history_0.html
ГовороваНД14951_history_0.html
ГурьяновАИ_history_0.html
МуравьевВВ15607_history_0.html
СелдаевАВ_history_0.html
ПацукТН_history_0.html
ПолозовНП_history_0.html
КозловИА18186_history_0.html
ГабоваАВ_history_0.html
МакароваТГ_history_0.html
ДороченковаАГ_history_0.html
ИоновИГ_history_0.html
КуликовАИ_history_0.html
ЗолотухинММ_history_0.html
КотовичФК_history_0.html
ФедоренкоТП_history_0.html
СмирныхОС_history_0.html
ЗайцевСА21576_history_0.html
АфанасьевАГ21654_history_0.html
ТовпекоВМ_history_0.html
ШевчукЛЯ_history_0.html
ЧучелинЛМ_history_0.html
БожковСВ_history_0.html
ВоблыйСА_history_0.html
СквирскийБД_history_0.html
СмеречинскаяРН_history_0.html
СыроватскаяЛВ_history_0.html
БожокАА5815_history_0.html
ШинкаренкоАЮ_history_0.html
ЮрчинкоТА_history_0.html
ЧервяковСИ7413_history_0.html
КлимовСМ_history_0.html
ОрловВГ_history_0.html
АриенкоГВ_history_0.html
ХазияхметоваТМ_history_0.html
СербиченкоЗП9714_history_0.html
ТишкинаНГ_history_0.html
ТарасенкоВМ_history_0.html
ЕмельяноваГЕ_history_0.html
КудрявцеваТВ_history_0.html
КлюшниковаВФ_history_0.html
ДударенкоАК_history_0.html
КирюхинаОС_history_0.html
ФроловВЯ1106_history_0.html
РахимовЛГ_history_0.html
ЮзиненеЛВ_history_0.html
ЗемцоваГА_history_0.html
КоршуковВП_history_0.html
КривенкоВН14048_history_0.html
ЛарченкоАП_history_0.html
ФомченкоВГ_history_0.html
РосейчукФМ_history_0.html
КривичИВ_history_0.html
СибирцеваМЛ_history_0.html
КоробковСВ17956_history_0.html
ПржеменецкаяВФ_history_0.html
АнащенкоВМ_history_0.html
ЯньшинВТ_history_0.html
КарпушкинАВ_history_0.html
МельниченкоВМ_history_0.html
СердюковЮВ_history_0.html
ГрафоваГТ_history_0.html
БеловаТИ_history_0.html
ВолошинаГИ21695_history_0.html
ВолошинаГИ21695_history_0.html
БашаВА2950_history_0.html
СидорчукЛИ_history_0.html
БанниковНЕ_history_0.html
КазаковВА3238_history_0.html
ВоронинаЛИ_history_0.html
ЦойЮП_history_0.html
СмирноваЛА5215_history_0.html
СтоматюкЮА_history_0.html
ВеликановМН_history_0.html
СковпинВК_history_0.html
БурковскаяГВ_history_0.html
КокоринЛМ_history_0.html
ЛюбушкинАА_history_0.html
МаликАС_history_0.html
БелоконьТА_history_0.html"""

In [ ]:
names_file = names_file + """САртемьеваОВ8037_history_0.html
НихаенкоЛВ_history_0.html
КолесниковАН9084_history_0.html
СоболевВИ9576_history_0.html
ТуговиковАА_history_0.html
КоролевАН19122_history_0.html
ТрофименкоАИ_history_0.html
КнязевЕВ10244_history_0.html
АлександровВА10463_history_0.html
КарасевКЛ_history_0.html
ГолованЕС_history_0.html
УмаровШохратМамед11936_history_0.html
ДьячковаТЯ11970_history_0.html
ВолковаТА12003_history_0.html
БуторинаТВ13514_history_0.html
ВороновичВА13566_history_0.html
РеуцкийОВ13667_history_0.html
ПелешокВА13984_history_0.html
ГулевскаяВМ13990_history_0.html
ПогосянОП13999_history_0.html
РойГГ14312_history_0.html
ХуторнойВА14380_history_0.html
КимВ14384_history_0.html
КузнецоваНВ14948_history_0.html
КорякинАГ_history_0.html
МищукЛИ15020_history_0.html
СученковаЗФ15184_history_0.html
ШустовАГ15707_history_0.html
СукачВС16008_history_0.html
КозюраСП16544_history_0.html
ЖмакинАБ17147_history_0.html
РужицкаяАА17521_history_0.html
КантемироваГВ17580_history_0.html
СкакуновВИ17747_history_0.html
ЧадовГЕ_history_0.html
ЛебединскаяТИ17801_history_0.html
КомароваВА17867_history_0.html
ГригоржевскаяВИ18003_history_0.html
НикатинАС_history_0.html
НестероваНИ18038_history_0.html
ФедотоваНВ18110_history_0.html
АвгустиновичГС18146_history_0.html
ТамгинаЛП18315_history_0.html
СлесаревскийАлександрЛеонидович18487_history_0.html
КотоваГИ18543_history_0.html
ХатыповИИ19756_history_0.html
ЗаводноваНМ19778_history_0.html
ФилимоноваРА19859_history_0.html
КучкинаРА19911_history_0.html
ПриваловаТП20146_history_0.html
ЛопатинАВ2734_history_0.html
КорольковаЛТ_history_0.html
ШаринГА_history_0.html
СтецуринаВГ383_history_0.html
МинаеваТС_history_0.html
ПланкинаЗС_history_0.html
КосинскаяНА_history_0.html
БуйволСВ_history_0.html
ГлянцевАИ_history_0.html
ВласоваНН_history_0.html
ЛогуновичСВ_history_0.html
ПодольскийЕЯ_history_0.html
КушнерГГ_history_0.html
ЛарькинаСА_history_0.html
ШокуровВВ_history_0.html
РомановБИ_history_0.html
МелеховАС_history_0.html
ЛевченкоВГ10964_history_0.html
ДьяченкоЛИ11119_history_0.html
ЯдринцевСИ11120_history_0.html
СеменоваМА12586_history_0.html
ГерасимовскийВЕ12935_history_0.html
РоманенкоАМ1362_history_0.html
ЗлаткинВМ13770_history_0.html
РоманцоваТП14882_history_0.html
ПетроваТМ16129_history_0.html
ПрусаковКВ17395_history_0.html
ШабаевВИ17710_history_0.html
АрбузовВА17928_history_0.html
ШпагинаГН18250_history_0.html
ЧертковВА18732_history_0.html
ПолтавецАВ18736_history_0.html
АлимоваНУ18741_history_0.html
АпеноваЛВ20276_history_0.html
ЕременкоАИ20684_history_0.html
ЕрмоленкоОВ21720_history_0.html
ЛяховаЛН22072_history_0.html
СахаровВВ2235_history_0.html
ЯскевичВИ22569_history_0.html
СухановГГ231_history_0.html
КовязинЛВ23804_history_0.html
МишенинаЛФ24392_history_0.html
МорозТП24932_history_0.html
ПотанинаРА25064_history_0.html
ЧетверяковВГ3237_history_0.html
ФеофановаЛМ3425_history_0.html
АбраменковАл5038_history_0.html
ТолкачеваИА5523_history_0.html
ЧудиновКВ610_history_0.html
НикифоровСМ6695_history_0.html
КосаревДС7215_history_0.html
ЖариковГВ7750_history_0.html
ПичуевСВ79_history_0.html
КонфедератоваТВ8140_history_0.html
БатдаловаВЛ8560_history_0.html
СкороходАТ907_history_0.html
КривецПВ9177_history_0.html
ДатковВН1412_history_0.html
СтецуренкоБГ77_history_0.html
ТрищенкоСА_history_0.html
СумскийНИ_history_0.html
ЧуеваЛГ_history_0.html
КостенкоВИ2817_history_0.html
РодионовАФ_history_0.html
ЦойАН4351_history_0.html
КалашниковИБ_history_0.html
МайстренкоВА_history_0.html
МасловКФ_history_0.html
СаржевскаяЛВ_history_0.html
ПакС5753_history_0.html
МаньЕИ6298_history_0.html
КирилюкАА6314_history_0.html
ДавидРИ_history_0.html
АллахвердиевТА_history_0.html
ГазинскийАМ_history_0.html
КистеньАИ_history_0.html
МихайликГС_history_0.html
КомоловВИ_history_0.html"""

In [ ]:
import sys
path_source_cod = r'/content/drive/MyDrive/Магистратура/Научная_работа/Parser/Source/'
sys.path.append(path_source_cod)

from constants import path_Epicrisis, path_HTML_history
from get_name import get_name_history
from get_cod import get_cod_history

import re
from os import walk, listdir
from bs4 import BeautifulSoup
from tqdm import tqdm
import datetime
import pandas as pd
import numpy as np

def get_Epicrisis_docs_all():
    table = []
    #z = 0
    print( "Загрузка документов вида \"Переводной эпикриз\",\"Этапный эпикриз\",\"Посмертный эпикриз\",\"Посмертный эпикриз\", начата" )
    for file in tqdm(listdir(path_HTML_history)):
        if '.html' not in file:
                continue
        if file not in names_file:
                continue

        with open(f'{path_HTML_history}/{file}', encoding='utf-8') as file:
                        src = file.read()
        soup = BeautifulSoup(src, 'lxml')
        name = None
        cod = None
        find = "Нет"

        name = get_name_history(soup)
        cod = get_cod_history(soup)

        src = src.replace("Больные с декомпенсацией хр. заболеваний: цирроз печени, ХОБЛ, хронический гемодиализ, диабет, онкология","")
        src = src.replace("Интерстициальное заболевание легких","")
        src = src.replace("Интерстициальное  заболевание легких","")
        src = src.replace("Тяжелые заболевания легких (включая пневмонию < 1 месяца назад)","")

        src = src.lower()
        html = re.findall( "хобл|заболевани[ея][ ]*легких",src )

        if( len(html) > 0 ):

          string_index = src.find(html[0])
          string = src[ string_index-20:string_index+20 ]

          table.append(
                {
                    "File Name": file.name,
                    "Name": str(name),
                    "Cod": str(cod),
                    "Find": "Да",
                    "Строка": string,
                }
          )
        else:
          table.append(
                {
                    "File Name": file.name,
                    "Name": str(name),
                    "Cod": str(cod),
                    "Find": "Нет",
                    "Строка": "-",
                }

        )
        #z = z +1
        #№if( z > 100 ):
        #  break

    table_df = pd.DataFrame( table )
    table_df.to_excel(path_Epicrisis+"_ХОБЛ_умершие2.xlsx",engine='xlsxwriter')
    print( "Загрузка документов вида \"Переводной эпикриз\",\"Этапный эпикриз\",\"Посмертный эпикриз\",\"Выписной эпикриз\" окончена, данные выгружены в таблицу : \n\t\t"+path_Epicrisis+"_ХОБЛ.xlsx" )
    return table_df

In [ ]:
import sys
path_source_cod = r'/content/drive/MyDrive/Магистратура/Научная_работа/Parser/Source/'
sys.path.append(path_source_cod)

from constants import path_Epicrisis, path_HTML_history
from get_name import get_name_history
from get_cod import get_cod_history

import re
from os import walk, listdir
from bs4 import BeautifulSoup
from tqdm import tqdm
import datetime
import pandas as pd
import numpy as np

def get_Epicrisis_docs_all():
    table = []
    #z = 0
    print( "Загрузка документов вида \"Переводной эпикриз\",\"Этапный эпикриз\",\"Посмертный эпикриз\",\"Посмертный эпикриз\", начата" )
    for file in tqdm(listdir(path_HTML_history)):
        if '.html' not in file:
                continue
        if file not in names_file:
                continue

        with open(f'{path_HTML_history}/{file}', encoding='utf-8') as file:
                        src = file.read()
        soup = BeautifulSoup(src, 'lxml')
        name = None
        cod = None
        find = "Нет"

        name = get_name_history(soup)
        cod = get_cod_history(soup)

        src = src.replace("Больные с декомпенсацией хр. заболеваний: цирроз печени, ХОБЛ, хронический гемодиализ, диабет, онкология","")
        src = src.replace("Интерстициальное заболевание легких","")
        src = src.replace("Интерстициальное  заболевание легких","")
        src = src.replace("Тяжелые заболевания легких (включая пневмонию < 1 месяца назад)","")

        src = src.lower()
        html = re.findall( " им |инфаркт миокарда|пикс",src )

        if( len(html) > 0 ):

          #string_index = src.find(html[0])
          #string = src[ string_index-20:string_index+20 ]

          table.append(
                {
                    "File Name": file.name,
                    "Name": str(name),
                    "Cod": str(cod),
                    "Find": "Да",
                    "Количество": len(html),
                }
          )
        else:
          table.append(
                {
                    "File Name": file.name,
                    "Name": str(name),
                    "Cod": str(cod),
                    "Find": "Нет",
                    "Количество": len(html),
                }

        )
        #z = z +1
        #№if( z > 100 ):
        #  break

    table_df = pd.DataFrame( table )
    table_df.to_excel(path_Epicrisis+"_ИМ.xlsx",engine='xlsxwriter')
    print( "Загрузка документов вида \"Переводной эпикриз\",\"Этапный эпикриз\",\"Посмертный эпикриз\",\"Выписной эпикриз\" окончена, данные выгружены в таблицу : \n\t\t"+path_Epicrisis+"_ХОБЛ.xlsx" )
    return table_df

In [ ]:
get_Epicrisis_docs_all()

In [ ]:
import sys
path_source_cod = r'/content/drive/MyDrive/Магистратура/Научная_работа/Parser/Source/'
sys.path.append(path_source_cod)

from constants import path_Epicrisis, path_HTML_history
from get_name import get_name_history
from get_cod import get_cod_history

import re
from os import walk, listdir
from bs4 import BeautifulSoup
from tqdm import tqdm
import datetime
import pandas as pd
import numpy as np

def get_Blocad_all():
    table = []
    #z = 0
    print( "Загрузка информации о \"А-В блокаде\" начата" )
    for file in tqdm(listdir(path_HTML_history)):

        if '.html' not in file:
                continue
        with open(f'{path_HTML_history}/{file}', encoding='utf-8') as file:
                        src = file.read()
        soup = BeautifulSoup(src, 'lxml')
        name = None
        cod = None
        find = "Нет"
        src = soup.text
        name = get_name_history(soup)
        cod = get_cod_history(soup)
        src = src.replace("&nbsp;"," ")
        src = src.lower()
        blocs = re.findall("а[- ]*в[ ]+блокад|a[- ]*v[ ]блокад",src)
        if( len(blocs) > 0  ):
          for bloc in blocs:
            index = src.find( bloc )
            if( index != -1 ):
              text = src[index-100:index+100]
              src = src[ index+100: ]
            else:
              text = "index = -1"
            table.append(
                    {
                        "File Name": file.name,
                        "Name": str(name),
                        "Cod": str(cod),
                        "Find": "Да",
                        "Text": text
                    }
            )
        else:
          table.append(
                {
                    "File Name": file.name,
                    "Name": str(name),
                    "Cod": str(cod),
                    "Find": "Нет",
                    "Text": "-",
                }

        )
        #z = z +1
        #if( z > 300 ):
        #  break

    table_df = pd.DataFrame( table )
    table_df.to_excel(path_Epicrisis+"_ХОБЛ.xlsx",engine='xlsxwriter')
    print( "Загрузка документов вида \"Переводной эпикриз\",\"Этапный эпикриз\",\"Посмертный эпикриз\",\"Выписной эпикриз\" окончена, данные выгружены в таблицу : \n\t\t"+path_Epicrisis+"_A-Б блокада.xlsx" )
    return table_df

In [ ]:
data_frame = get_Blocad_all()

100%|██████████| 15354/15354 [3:50:23<00:00,  1.11it/s]


Загрузка документов вида "Переводной эпикриз","Этапный эпикриз","Посмертный эпикриз","Выписной эпикриз" окончена, данные выгружены в таблицу : 
		/content/drive/MyDrive/Магистратура/Научная работа/DataSetes/Histores/Docs/Эпикризы.xlsx_A-Б блокада.xlsx


In [ ]:
data_frame = data_frame.loc[(data_frame["Find"]=="Да")]
print(data_frame)

In [ ]:
!pip install ufal.udpipe
!pip install umap-learn
!pip install umap-learn[plot]
!pip install pymorphy2

import ufal.udpipe

from tqdm.auto import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from google.colab import drive
import umap
import umap.plot
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pymorphy2

drive.mount('/content/drive')

import pandas as pd

umap.plot.output_notebook()

In [ ]:


class TokenizerNormForm(object):
    def __init__(self):
        self.morph = pymorphy2.MorphAnalyzer()
        self.model = ufal.udpipe.Model.load("/content/drive/MyDrive/Магистратура/Компьютерная лингвистика/lab1/russian-syntagrus-ud-2.5-191206.udpipe")
        self.tokenizer = self.model.newTokenizer(self.model.DEFAULT)
        self.sentence = ufal.udpipe.Sentence()
        self.error = ufal.udpipe.ProcessingError()




    def __call__(self, text):
      words = []
      av = ["av","ав"]

      stop_list = get_stop_list("/content/drive/MyDrive/Магистратура/Компьютерная лингвистика/texts/stopwords-ru.txt.txt")
      stop_list.append( "." )
      stop_list.append( ":" )
      stop_list.append( "к" )
      stop_list.append( "(" )
      stop_list.append( ")" )
      stop_list.append( "\"" )
      stop_list.append( "," )
      stop_list.append( "-" )

      self.tokenizer.setText(text)
      while self.tokenizer.nextSentence( self.sentence,  self.error):
              self.model.tag( self.sentence,  self.model.DEFAULT)
              #self.model.parse( self.sentence,  self.model.DEFAULT)
              for i, word in enumerate(self.sentence.words):
                if(
                    (   word.upostag == "VERB"
                    or  word.upostag == "NOUN"
                    or  word.upostag == "ADJF"
                    or  word.upostag == "ADJS"
                    or  word.upostag == "PRTF"
                    or  word.upostag == "PRTS"
                    or  word.upostag == "GRND")
                    and ( not word.form in stop_list )
                    and ( i+1<len(self.sentence.words) and not self.sentence.words[i+1].form in stop_list )
                    ):
                  #
                  #if( w == "ав" ):
                  #  print(word.upostag)
                  #print(word.form=="ав")
                  if( i+1<len(self.sentence.words) ):
                    if( not word.form in av ):
                      w1 = self.morph.parse(word.form)[0].normal_form
                    else:
                      w1 = word.form

                    if( not word.form in av ):
                      w2 = self.morph.parse(self.sentence.words[i+1].form)[0].normal_form
                    else:
                      w2 = self.sentence.words[i+1].form
                    words.append(w1+" "+w2)
      return words

def get_stop_list(path):
    with open(path, encoding="utf-8") as news_file:
      stop_list = [n for n in news_file.read().split("\n")]
    return stop_list

def get_vocabulary( vocabulary_old , vector ):
    vocabulary = []
    for word,index in vocabulary_old.items():
        sum = 0
        for doc in vector:
          sum += doc[index]
        vocabulary.append( { "Word":word, "Sum": sum } )
    dataframe = pd.DataFrame(vocabulary,columns=["Word","Sum"])
    return dataframe

In [ ]:
stop_list = get_stop_list("/content/drive/MyDrive/Магистратура/Компьютерная лингвистика/texts/stopwords-ru.txt.txt")
data_frame

In [ ]:
")" in stop_list

In [ ]:
vectorizerCV = CountVectorizer( tokenizer=TokenizerNormForm(),
                                analyzer='word')
stop_list.append( ")" )
stop_list.append( "\"" )
stop_list.append( "\"" )
stop_list.append( "\"" )
stop_list.append( "\"" )

vectorizerCV.stop_words = stop_list
vectorCV = vectorizerCV.fit_transform(data_frame["Text"].values)

vocabulary = get_vocabulary(vectorizerCV.vocabulary_,vectorCV.toarray())
vocabulary = vocabulary.sort_values(by="Sum",ascending=False)
vocabulary.to_excel(path_Epicrisis+"___Словарь___.xlsx",engine='xlsxwriter')
print(vocabulary)
plt.plot(range(len(vocabulary['Sum'])),vocabulary['Sum'])
plt.show()



In [ ]:
# Установка spaCy
!pip3 install spacy

# Загрузка модели для анализа английского языка
!python3 -m spacy download ru_core_news_sm

# Установка textacy
!pip3 install -U textacy


In [ ]:
import spacy


text = data_frame["Text"].values[0]
text = "Он пошел гулять. Велосипед упал."
print(text)
nlp = spacy.load('ru_core_news_sm')
doc = nlp(text)
print(doc)
for entity in doc.ents:
    print(f"{entity.text} ({entity.label_})")


Он пошел гулять. Велосипед упал.
Он пошел гулять. Велосипед упал.


In [ ]:
import pandas as pd
import re
from tqdm import tqdm
path_DataSet_Perv = r'/content/drive/MyDrive/Магистратура/Научная работа/DataSetes/Histores/Docs/Первичный врачебный осмотр.xlsx'
path_DataSet_Epic = r'/content/drive/MyDrive/Магистратура/Научная работа/DataSetes/Histores/Docs/Эпикризы.xlsx'
path_DataSet = path_DataSet_Epic
data = pd.read_excel( path_DataSet )

In [ ]:
data["Количесто ИМ"] = None

for index, row in tqdm(data.iterrows()):
    string = row['HTML']
    #print(string)
    if not type(string) is float:
      string = string.lower()
      string = string.replace( "&nbsp;", " " )

      html = re.findall( " им |инфаркт миокарда|пикс",src )

    data["Ишемический инсульт"][index] = isIsh
    data["Геморрагический инсульт"][index] = isGem
    data["Инсульт"][index] = isInsult
    data["Инсульт отрицает"][index] = noInsult
data.to_excel(path_DataSet+"_Инсульт.xlsx",engine='xlsxwriter')


0it [00:00, ?it/s]<ipython-input-35-89f271789ad5>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Ишемический инсульт"][index] = isIsh
<ipython-input-35-89f271789ad5>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Геморрагический инсульт"][index] = isGem
<ipython-input-35-89f271789ad5>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Инсульт"][index] = isInsult
<ipython-input-35-89f271789ad5>:50: SettingWithCopy

In [ ]:
data.loc[ data["Ишемический инсульт"] == "Да" ]

In [ ]:
import sys
path_source_cod = r'/content/drive/MyDrive/Магистратура/Научная_работа/Parser/Source/'
sys.path.append(path_source_cod)

from constants import path_Epicrisis, path_HTML_history
from get_name import get_name_history
from get_cod import get_cod_history

import re
from os import walk, listdir
from bs4 import BeautifulSoup
from tqdm import tqdm
import datetime
import pandas as pd
import numpy as np

def get_Epicrisis_docs_all():
    table = []
    #z = 0
    print( "Загрузка документов вида \"Переводной эпикриз\",\"Этапный эпикриз\",\"Посмертный эпикриз\",\"Посмертный эпикриз\", начата" )
    for file in tqdm(listdir(path_HTML_history)):
        if '.html' not in file:
                continue
        with open(f'{path_HTML_history}/{file}', encoding='utf-8') as file:
                        src = file.read()
        soup = BeautifulSoup(src, 'lxml')
        name = None
        cod = None
        find = "Нет"

        name = get_name_history(soup)
        cod = get_cod_history(soup)

        #src = src.replace("заболеваний: цирроз печени, ХОБЛ, хронический гемодиализ","")
        #src = src.replace("руктивные заболевания легких (ХОБЛ)","")

        src = src.replace("SpO2","SpOTwo")
        src = src.replace("Sp02","SpOTwo")
        src = src.replace("SpО2","SpOTwo")
        src = src.lower()


        html = re.findall( "spotwo[ -]*\d+",src )
        if( len(html)>0 ):

          SpO2 = html[0]
          SpO2 = re.findall( "\d+",SpO2 )
          SpO2 = int(SpO2[0])
          table.append(
                {
                    "File Name": file.name,
                    "Name": str(name),
                    "Cod": str(cod),
                    "Find": "Да",
                    "SpO2": SpO2,
                }
          )
        else:
          table.append(
                {
                    "File Name": file.name,
                    "Name": str(name),
                    "Cod": str(cod),
                    "Find": "Нет",
                    "SpO2": "-",
                }

        )
        #z = z +1
        #if( z > 100 ):
        #  break

    table_df = pd.DataFrame( table )
    table_df.to_excel(path_Epicrisis+"_SpO2.xlsx",engine='xlsxwriter')
    print( "Загрузка документов вида \"Переводной эпикриз\",\"Этапный эпикриз\",\"Посмертный эпикриз\",\"Выписной эпикриз\" окончена, данные выгружены в таблицу : \n\t\t"+path_Epicrisis+"_ХОБЛ.xlsx" )
    return table_df

In [ ]:
get_Epicrisis_docs_all()

In [ ]:
import sys
path_source_cod = r'/content/drive/MyDrive/Магистратура/Научная_работа/Parser/Source/'
sys.path.append(path_source_cod)

from constants import path_Epicrisis, path_HTML_history
from get_name import get_name_history
from get_cod import get_cod_history

import re
from os import walk, listdir
from bs4 import BeautifulSoup
from tqdm import tqdm
import datetime
import pandas as pd
import numpy as np

def get_Epicrisis_docs_all():
    table = []
    z = 0
    print( "Загрузка документов вида \"Переводной эпикриз\",\"Этапный эпикриз\",\"Посмертный эпикриз\",\"Посмертный эпикриз\", начата" )
    for file in tqdm(listdir(path_HTML_history)):
        if '.html' not in file:
                continue
        #if file not in names_file:
        #        continue

        with open(f'{path_HTML_history}/{file}', encoding='utf-8') as file:
                        src = file.read()
        soup = BeautifulSoup(src, 'lxml')
        name = None
        cod = None
        find = "Нет"

        name = get_name_history(soup)
        cod = get_cod_history(soup)
        src = src.lower()

        #src = src.replace("td width=\"95%\">Отек легких","")
        src = src.replace(">отек легких, обуслов","")
        src = src.replace(">отек легких, наиболе","")

        html = re.findall( "от[её]к[а ]*легких|интерстициальн[ыйого ]*от[её]к[а]*|интерстициальн[ыйого ]*от[её]к",src )
        html_no = re.findall( "отека легких нет|отека нет",src )

        if( len(html) > 0 ):

          string_index = src.find(html[0])
          string = src[ string_index-20:string_index+20 ]

          if( len(html_no)>0 ):
            res = "Нет"
          else:
            res = "Да"

          table.append(
                {
                    "File Name": file.name,
                    "Name": str(name),
                    "Cod": str(cod),
                    "Find": res,
                    "Строка":string,
                    "Количество": len(html),
                }
          )
        else:
          table.append(
                {
                    "File Name": file.name,
                    "Name": str(name),
                    "Cod": str(cod),
                    "Find": "Нет",
                    "Строка":"-",
                    "Количество": len(html),
                }

        )
        #z = z +1
        #if( z > 150 ):
        #  break

    table_df = pd.DataFrame( table )
    table_df.to_excel(path_Epicrisis+"_ИМ.xlsx",engine='xlsxwriter')
    print( "Загрузка документов вида \"Переводной эпикриз\",\"Этапный эпикриз\",\"Посмертный эпикриз\",\"Выписной эпикриз\" окончена, данные выгружены в таблицу : \n\t\t"+path_Epicrisis+"_Отек_Легких.xlsx" )
    return table_df

In [ ]:
get_Epicrisis_docs_all()

In [ ]:

import sys
path_source_cod = r'/content/drive/MyDrive/Магистратура/Научная_работа/Parser/Source/'
sys.path.append(path_source_cod)

from constants import path_Epicrisis, path_HTML_history
from get_name import get_name_history
from get_cod import get_cod_history

import re
from os import walk, listdir
from bs4 import BeautifulSoup
from tqdm import tqdm
import datetime
import pandas as pd
import numpy as np

def get_PrimaryMedicalExamination_doc(text):
    text_reserv = text
    res = [  ]
    doc = "<h4>Первичный врачебный осмотр</h4>"

    while ( text.find(doc) != -1  ):
      index_beging = text.find(doc)
      if( index_beging == -1 ):
        return None

      text = text[index_beging:]

      iter_line = re.finditer("<hr>",text)
      index_line = []
      for i in iter_line:
        index_line.append( i.start() )

      if( len(index_line)>=2 ):
        res.append( text[:index_line[1]] )
        text = text[index_line[1]:]
      else:
        res.append( None )
        text = text[index_line[0]:]

    return res

def get_Epicrisis_docs_all():
    table = []
    z = 0
    print( "Загрузка документов вида \"Переводной эпикриз\",\"Этапный эпикриз\",\"Посмертный эпикриз\",\"Посмертный эпикриз\", начата" )
    for file in tqdm(listdir(path_HTML_history)):
        if '.html' not in file:
                continue
        #if file not in names_file:
        #        continue

        with open(f'{path_HTML_history}/{file}', encoding='utf-8') as file:
                        src = file.read()
        soup = BeautifulSoup(src, 'lxml')
        name = None
        cod = None
        print(file.name)
        name = get_name_history(soup)
        cod = get_cod_history(soup)
        src_tmp = src
        src = src.lower()

        #
        #src = src.replace(">отек легких, обуслов","")
        #src = src.replace(">отек легких, наиболе","")

        city = re.findall( "постоянное место жительства.*тел",src )
        isVladivostoc = "Нет"
        if( len(city) > 0 ):
          city_str = city[0]
          if( re.findall("владивосток",city_str) ):
            isVladivostoc = "Да"
        else:
          city_str = "-"

        src = src.replace("до проведения тромболизиса более 4,5 часов","")
        src = src.replace("системный внутривенный тромболизис не позднее 40 минут","")

        tromb = re.findall( "тромболиз",src )
        no_tromb = re.findall( "тромболизис не проведен",src )
        tromb_str = "-"
        if( len(tromb)>0 and len(no_tromb)==0 ):
          index = src.find(tromb[0])
          tromb_str = src[index-40:index+40]
          tromb = "Да"
        else:
          tromb = "Нет"



        PrimaryMedicalExamination = get_PrimaryMedicalExamination_doc(src_tmp)
        data_beging = "-"
        y = "1900"
        if( len(PrimaryMedicalExamination)>0 ):
          html = PrimaryMedicalExamination[0]
          html = html.lower()
          html = html.replace("&nbsp;","")
          #if( file.name == "/content/drive/MyDrive/Магистратура/Научная работа/DataSetes/Histores/Histores_HTML/ЧучелинЛМ_history_0.html" ):
          #  print(html)
          if( re.search( "дата и время осмотра <i>\d\d.\d\d.\d\d\d\d,\d\d.\d\d",html ) ):
            data_time = re.findall( "дата и время осмотра <i>\d\d.\d\d.\d\d\d\d,\d\d.\d\d",html )[0]
            data = re.findall( "\d\d.\d\d.\d\d\d\d",data_time )[0]
            data = data.replace( "/","." )
            data = data.replace( "\\","." )
            data = data.replace( "-","." )
            data = data.replace( ",","." )
            y = re.findall( "\d\d\d\d",data )[0]
            data_time = data_time.replace(data,"")
            time = re.findall( "\d\d.\d\d",data_time )[0]
            time = time.replace( ".",":" )
            time = time.replace( "-",":" )
            time = time.replace( ",",":" )
            data_osmotr_str = data
            data_osmotr = datetime.datetime.strptime( data+" "+time, '%d.%m.%Y %H:%M')

          try:
            if( re.search( "\d\d.\d\d.\d\d\d\d[а-яё ,]+\d\d.\d\d[а-яё ,]+ухудшен[ияее ]+состоян[ияе]+|\d\d.\d\d.\d\d\d\d[а-яё ,]+\d\d.\d\d[а-яё ,]+бол[ейи]+",html ) ):
              data_time = re.findall( "\d\d.\d\d.\d\d\d\d[а-яё ,]+\d\d.\d\d[а-яё ,]+ухудшен[ияее ]+состоян[ияе]+|\d\d.\d\d.\d\d\d\d[а-яё ,]+\d\d.\d\d[а-яё ,]+бол[ейи]+",html )[0]
              data = re.findall( "\d\d.\d\d.\d\d\d\d",data_time )[0]
              data = data.replace( "/","." )
              data = data.replace( "\\","." )
              data = data.replace( "-","." )
              data = data.replace( ",","." )
              data_time = data_time.replace(data,"")
              time = re.findall( "\d\d.\d\d",data_time )[0]
              time = time.replace( ".",":" )
              time = time.replace( "-",":" )
              time = time.replace( ",",":" )
              data_beging = datetime.datetime.strptime( data+" "+time, '%d.%m.%Y %H:%M')
            elif( re.search( "\d.\d\d[\., а-яё]*\d\d:\d\d[\., а-яё]*возникла[\., а-яё]*боль",html ) ):
              data_time = re.findall( "\d.\d\d[\., а-яё]*\d\d:\d\d[\., а-яё]*возникла[\., а-яё]*боль",html )[0]
              data = re.findall( "\d.\d\d",data_time )[0]
              data = data.replace( "/","." )
              data = data.replace( "\\","." )
              data = data.replace( "-","." )
              data = data.replace( ",","." )
              data = "0"+data+"."+y
              data_time = data_time.replace(data,"")
              time = re.findall( "\d\d.\d\d",data_time )[0]
              time = time.replace( ".",":" )
              time = time.replace( "-",":" )
              time = time.replace( ",",":" )
              data_beging = datetime.datetime.strptime( data+" "+time, '%d.%m.%Y %H:%M')
            elif( re.search( "\d\d.\d\d.\d\d[а-яё ,]+\d\d[а-яё ,\.]+ухудшен[ияее ]+состоян[ияе]+|\d\d.\d\d.\d\d[а-яё ,]+\d\d[а-яё ,\.]+бол[ейи]+|ухудшен[а-яё ,]+\d\d[а-яё ,]+\d\d.\d\d.\d\d",html ) ):
              data_time = re.findall( "\d\d.\d\d.\d\d[а-яё ,]+\d\d[а-яё ,\.]+ухудшен[ияее ]+состоян[ияе]+|\d\d.\d\d.\d\d[а-яё ,]+\d\d[а-яё ,\.]+бол[ейи]+|ухудшен[а-яё ,]+\d\d[а-яё ,]+\d\d.\d\d.\d\d",html )[0]
              data = re.findall( "\d\d.\d\d.\d\d",data_time )[0]
              data = data.replace( "/","." )
              data = data.replace( "\\","." )
              data = data.replace( "-","." )
              data = data.replace( ",","." )
              data_time = data_time.replace(data,"")
              time = re.findall( "\d\d",data_time )[0]
              time = time.replace( ".",":" )
              time = time.replace( "-",":" )
              time = time.replace( ",",":" )
              time = time +":00"
              data_beging = datetime.datetime.strptime( data+" "+time, '%d.%m.%y %H:%M')
            elif( re.search( "боль[а-яё ,-]+\d:\d\d[а-яё ,-]+\d\d.\d\d",html ) ):
              data_time = re.findall( "боль[а-яё ,-]+\d.\d\d[а-яё ,-]+\d\d.\d\d",html )[0]
              time = re.findall( "\d:\d\d",data_time )[0]
              time = time.replace( ".",":" )
              time = time.replace( "-",":" )
              time = time.replace( ",",":" )
              data_time = data_time.replace(time,"")
              time = "0"+time

              data = re.findall( "\d\d.\d\d",data_time )[0]
              data = data.replace( "/","." )
              data = data.replace( "\\","." )
              data = data.replace( "-","." )
              data = data.replace( ",","." )
              data_time = data_time.replace(data,"")
              data = data+"."+y
              data_beging = datetime.datetime.strptime( data+" "+time, '%d.%m.%Y %H:%M')
            elif( re.search( "сегодня[а-яё ,-]+\d\d.\d\d[а-яё ,-]+болев",html ) ):
              data_time = re.findall( "сегодня[а-яё ,-]+\d\d.\d\d[а-яё ,-]+болев",html )[0]
              time = re.findall( "\d\d.\d\d",data_time )[0]
              time = time.replace( ".",":" )
              time = time.replace( "-",":" )
              time = time.replace( ",",":" )
              data = data_osmotr_str
              data_beging = datetime.datetime.strptime( data+" "+time, '%d.%m.%Y %H:%M')

            elif( re.search( "сегодня[а-яё ,-]+\d.\d\d[а-яё ,-]+бол",html ) ):
              data_time = re.findall( "сегодня[а-яё ,-]+\d.\d\d[а-яё ,-]+бол",html )[0]
              time = re.findall( "\d.\d\d",data_time )[0]
              time = time.replace( ".",":" )
              time = time.replace( "-",":" )
              time = time.replace( ",",":" )
              time = "0"+time
              data = data_osmotr_str
              data_beging = datetime.datetime.strptime( data+" "+time, '%d.%m.%Y %H:%M')

            elif( re.search( "\d\d[\.\-/\\,]\d\d[\.\-/\\,]\d\d\d\d[а-яё ,]+\d[а-яё ,]+ухудшен[ияее ]+состоян[ияе]+|\d\d[\.\-/\\,]\d\d[\.\-/\\,]\d\d\d\d[а-яё ,]+\d[а-яё ,]+бол[ейиь]+",html ) ):
              data_time = re.findall( "\d\d[\.\-/\\,]\d\d[\.\-/\\,]\d\d\d\d[а-яё ,]+\d[а-яё ,]+ухудшен[ияее ]+состоян[ияе]+|\d\d[\.\-/\\,]\d\d[\.\-/\\,]\d\d\d\d[а-яё ,]+\d[а-яё ,]+бол[ейиь]+",html )[0]
              data = re.findall( "\d\d[\.\-/\\,]\d\d[\.\-/\\,]\d\d\d\d",data_time )[0]
              data = data.replace( "/","." )
              data = data.replace( "\\","." )
              data = data.replace( "-","." )
              data = data.replace( ",","." )
              data_time = data_time.replace(data,"")
              time = re.findall( "\d",data_time )[0]
              time = time.replace( ".",":" )
              time = time.replace( "-",":" )
              time = time.replace( ",",":" )
              time = "0" + time + ":00"
              data_beging = datetime.datetime.strptime( data+" "+time, '%d.%m.%Y %H:%M')
            if( re.search( "\d\d[\.\-/\\,]\d\d[\.\-/\\,]\d\d[а-яё ,\.]+\d\d.\d\d[а-яё ,\.]+ухудшен[ияее ]+состоян[ияе]+|\d\d[\.\-/\\,]\d\d[\.\-/\\,]\d\d[а-яё ,\.]+\d\d.\d\d[а-яё ,\.]+бол[ейиь]+",html ) ):
              data_time = re.findall( "\d\d[\.\-/\\,]\d\d[\.\-/\\,]\d\d[а-яё ,\.]+\d\d.\d\d[а-яё ,\.]+ухудшен[ияее ]+состоян[ияе]+|\d\d[\.\-/\\,]\d\d[\.\-/\\,]\d\d[а-яё ,\.]+\d\d.\d\d[а-яё ,\.]+бол[ейиь]+",html )[0]
              data = re.findall( "\d\d[\.\-/\\,]\d\d[\.\-/\\,]\d\d",data_time )[0]
              data = data.replace( "/","." )
              data = data.replace( "\\","." )
              data = data.replace( "-","." )
              data = data.replace( ",","." )
              data_time = data_time.replace(data,"")
              time = re.findall( "\d\d.\d\d",data_time )[0]
              time = time.replace( ".",":" )
              time = time.replace( "-",":" )
              time = time.replace( ",",":" )
              data_beging = datetime.datetime.strptime( data+" "+time, '%d.%m.%y %H:%M')
            elif( re.search( "\d[\.:]\d\d[а-яё ,\.]+\d\d.\d\d.\d\d[а-яё ,]+бол[ияеь ]+",html ) ):

              data_time = re.findall( "\d[\.:]\d\d[а-яё ,\.]+\d\d.\d\d.\d\d[а-яё ,]+бол[ияеь ]+",html )[0]

              time = re.findall( "\d.\d\d",data_time )[0]
              time = time.replace( ".",":" )
              time = time.replace( "-",":" )
              time = time.replace( ",",":" )
              data_time = data_time.replace(time,"")
              time = "0"+time

              data = re.findall( "\d\d.\d\d.\d\d",data_time )[0]
              data = data.replace( "/","." )
              data = data.replace( "\\","." )
              data = data.replace( "-","." )
              data = data.replace( ",","." )
              data_time = data_time.replace(data,"")
              data_beging = datetime.datetime.strptime( data+" "+time, '%d.%m.%y %H:%M')
          except Exception as e:
              data_beging = "-"

          ras = "-"
          if( data_beging != "-" and data_osmotr != "-" ):
              ras = (data_osmotr - data_beging).seconds/60/60

        if( tromb == "Да" ):
          bal = 0
        elif( ras!="-" and ras < 4 ):
          bal = 0
        elif( tromb == "Нет" and isVladivostoc == "Нет" ):
          bal = 1
        elif( isVladivostoc == "Да" and ras == "-" ):
          bal = 0
        else:
          bal = 1

        table.append(
                {
                    "File Name": file.name,
                    "Name": str(name),
                    "Cod": str(cod),
                    "Город строка":city_str,
                    "Владивосток":isVladivostoc,
                    "Тромболизис строка":tromb_str,
                    "Тромболизис":tromb,
                    "Дата начала болевого сентдорома":data_beging,
                    "Дата осмотра":data_osmotr,
                    "Разница дат":ras,
                    "Первичный осмотр":html,
                    "Балл":bal,
                })

        #z = z +1
        #if( z > 100 ):
        #  break

    table_df = pd.DataFrame( table )
    table_df.to_excel(path_Epicrisis+"_ИМ.xlsx",engine='xlsxwriter')
    print( "Загрузка документов вида \"Переводной эпикриз\",\"Этапный эпикриз\",\"Посмертный эпикриз\",\"Выписной эпикриз\" окончена, данные выгружены в таблицу : \n\t\t"+path_Epicrisis+"_Время.xlsx" )
    return table_df

In [ ]:
get_Epicrisis_docs_all()

In [ ]:
def get_Epicrisis_docs_all():
    table = []
    z = 0
    print( "Загрузка документов вида \"Переводной эпикриз\",\"Этапный эпикриз\",\"Посмертный эпикриз\",\"Посмертный эпикриз\", начата" )
    for file in tqdm(listdir(path_HTML_history)):
        if '.html' not in file:
                continue
        #if file not in names_file:
        #        continue

        with open(f'{path_HTML_history}/{file}', encoding='utf-8') as file:
                        src = file.read()
        soup = BeautifulSoup(src, 'lxml')
        name = None
        cod = None
        print(file.name)
        name = get_name_history(soup)
        cod = get_cod_history(soup)
        html = soup.text
        html = html.lower()
        html.replace("&nbsp;"," ")
        killip_str_mas = re.findall( "killip|kilip|осн",html )
        killip = "-"
        killip_str = "-"
        for killip_str in killip_str_mas:

          string_index = html.find(killip_str)
          killip_str = html[ string_index-20:string_index+20 ]

          killip_str = killip_str.replace( "iv","4" )

          killip_str = killip_str.replace( " ш "," 3 " )
          killip_str = killip_str.replace( " щ "," 3 " )
          killip_str = killip_str.replace( "iii","3" )
          killip_str = killip_str.replace( "lll","3" )
          killip_str = killip_str.replace( "lll","3" )

          killip_str = killip_str.replace( " п "," 2 " )
          killip_str = killip_str.replace( "ii"," 2 " )

          killip_str = killip_str.replace( "i ", "1 " )
          killip_str = killip_str.replace( " i"," 1" )
          killip_str = killip_str.replace( " i"," 1" )
          killip_str = killip_str.replace( "killipi","killip 1" )

          killip = re.findall( "killip[ -]*\d-\d|осн[ -]*\d-\d|\d-\d[ -]*killip",killip_str)
          if( len(killip)>0 ):
            killip = max([int( re.findall( "\d",killip[0] )[0] ),int( re.findall( "\d",killip[0] )[1] )])
          elif( re.search( "killip[ -]*\d|осн[ -]*\d|\d[ -]*killip",killip_str) ):
            killip = re.findall( "killip[ -]*\d|осн[ -]*\d|\d[ -]*killip",killip_str)
            killip = int( re.findall( "\d",killip[0] )[0] )
          else:
            killip = "-"
          if( killip != "-" ):
            break

        table.append(
                {
                    "File Name": file.name,
                    "Name": str(name),
                    "Cod": str(cod),
                    "ОСН по Killip строка":killip,
                    "ОСН по Killip":killip_str,
                })

        #z = z +1
        #if( z > 1000 ):
        #  break

    table_df = pd.DataFrame( table )
    table_df.to_excel(path_Epicrisis+"_killip.xlsx",engine='xlsxwriter')
    print( "Загрузка документов вида \"Переводной эпикриз\",\"Этапный эпикриз\",\"Посмертный эпикриз\",\"Выписной эпикриз\" окончена, данные выгружены в таблицу : \n\t\t"+path_Epicrisis+"_Время.xlsx" )
    return table_df

In [ ]:
get_Epicrisis_docs_all()

Output hidden; open in https://colab.research.google.com to view.